In [ ]:
#@title License
license = """
Copyright 2022 DeepMind Technologies Limited

Licensed under the Apache License, Version 2.0 (the "License"); you may not
use this file except in compliance with the License. You may obtain a copy of
the License at https://www.apache.org/licenses/LICENSE-2.0. Unless required by
applicable law or agreed to in writing, software distributed under the License
is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied. See the License for the specific language
governing permissions and limitations under the License.
"""
print(license)

# Dramatron

**Piotr Mirowski, Kory Mathewson, Juliette Love, Jaylen Pittman**, based on an original prototype by **Richard Evans**.

DeepMind, 2022

Dramatron is a script writing tool that leverages large language models. It uses hierarchical generation to ensure long range consistency across the entire script. The work was published as a pre-print in Mirowski, Mathewson et al. (2022) "[Co-Writing Screenplays and Theatre Scripts with Language Models: An Evaluation by Industry Professionals](https://arxiv.org/abs/2209.14958)".

![picture](https://github.com/deepmind/dramatron/raw/main/dramatron_logo.png)

## Citing this work (paper and code):
```bibtex
@article{mirowski2022cowriting,
  title={Co-Writing Screenplays and Theatre Scripts with Language Models: An Evaluation by Industry Professionals},
  author={Mirowski, Piotr and Mathewson, Kory W and Pittman, Jaylen and Evans, Richard},
  journal={arXiv preprint arXiv:2209.14958},
  year={2022}
}
```

# Setup

Run this set of cells to install and import libraries, define the code, set the parameters of Dramatron and Perspective API (moderation, optional) and the prompt sets.

In [ ]:
#@title Google PaLM 2 only: Install dependencies (Vertex AI) and restart the colab runtime

#@markdown The Python SDK for the Palm 2 API and the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:
!pip install -q -U google-generativeai
# !pip install google-cloud-aiplatform --upgrade --user

In [ ]:
#@title Imports

#@markdown Run this cell to import the required Python libraries.

import ipywidgets as widgets
from IPython.display import HTML, display

import datetime
import difflib
import json
import re
import requests
import sys
import time
from typing import Dict, List, NamedTuple, Optional, Union

import collections
from google.colab import files

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

print('Imports imported successfully!')

In [ ]:
#@title Dramatron hyperparameters { run: "auto" }

#@markdown Run this cell to set the model hyperparameters. The cell will auto-run if you change them. We tested the model with these default values.

#@markdown Default seed for generation (default: 1)
DEFAULT_SEED =  1  #@param {type:"integer"}
#@markdown Sampling top-p probability (default: 0.9) and temperature (default 1.0)
SAMPLING_PROB =  0.9  #@param {type:"slider", min:0.8, max:1.0, step:0.01}
SAMPLING_TEMP =  1.  #@param {type:"slider", min:0.8, max:1.0, step:0.01}
#@markdown Max length for the generated title, place description and others, in tokens (defaults: 64, 128 and 511 respectively)
SAMPLE_LENGTH_TITLE = 64 #@param [64, 128, 256, 511]
SAMPLE_LENGTH_PLACE = 128 #@param [128, 256, 511]
SAMPLE_LENGTH = 511 #@param [128, 256, 511]
#@markdown Max lengths during repeated sampling, in case `<end>` is not found (default: 2048)
MAX_PARAGRAPH_LENGTH_CHARACTERS = 1024 #@param [511, 1024, 2048, 4096]
MAX_PARAGRAPH_LENGTH_SCENES = 1024 #@param [511, 1024, 2048, 4096]
MAX_PARAGRAPH_LENGTH = 1024 #@param [511, 1024, 2048, 4096]
#@markdown Unavailable API: max number of retries before giving up (default: 10)
MAX_RETRIES = 10 #@param {type:"slider", min:1, max:20, step:1}
#@markdown Loop detection: max number of repetitions before resampling, and number of attempts to get out of the loop (default: 3)
MAX_NUM_REPETITIONS = 3 #@param {type:"slider", min:1, max:10, step:1}
MAX_NUM_ATTEMPTS_GET_OUT_OF_LOOP = 3 #@param {type:"slider", min:1, max:10, step:1}

print('Dramatron hyperparameters set.')


In [ ]:
#@title Dramatron code

#@markdown This cell contains Dramatron's code. You need to run it only once.

# ------------------------------------------------------------------------------
# Script markers
# ------------------------------------------------------------------------------

# END_MARKER = '<end>'
# STOP_MARKER = '<stop>'
# CHARACTER_MARKER = '<character>'
# DESCRIPTION_MARKER = '<description>'
# SCENES_MARKER = '<scenes>'
# DIALOG_MARKER = '<dialog>'
# TITLE_ELEMENT = 'Title: '
# CHARACTERS_ELEMENT = 'Characters: '
# DESCRIPTION_ELEMENT = 'Description: '
# PLACE_ELEMENT = 'Place: '
# PLOT_ELEMENT = 'Plot element: '
# PREVIOUS_ELEMENT = 'Previous beat: '
# SUMMARY_ELEMENT = 'Summary: '
# BEAT_ELEMENT = 'Beat: '
END_MARKER = '**END**'
STOP_MARKER = '\n'
CHARACTER_MARKER = '**Character:** '
DESCRIPTION_MARKER = '**Description:** '
SCENES_MARKER = '**Scenes:**'
DIALOG_MARKER = '**Dialog:**'
LOGLINE_MARKER = "**Logline:** "
TITLE_ELEMENT = 'Title: '
CHARACTERS_ELEMENT = 'Characters: '
DESCRIPTION_ELEMENT = 'Description: '
PLACE_ELEMENT = 'Place: '
PLOT_ELEMENT = 'Plot element: '
PREVIOUS_ELEMENT = 'Previous beat: '
SUMMARY_ELEMENT = 'Summary: '
BEAT_ELEMENT = 'Beat: '
LOGLINE_ELEMENT = "Logline: "

# ------------------------------------------------------------------------------
# Dramatron script entities
# ------------------------------------------------------------------------------


class Title(NamedTuple):
  """Title class."""

  title: str

  @classmethod
  def from_string(cls, text: str):
    title = extract_elements(text, TITLE_ELEMENT, END_MARKER)[0]
    return cls(title)

  def to_string(self):
    s = ''
    s += TITLE_ELEMENT + self.title
    s += END_MARKER
    return s


def get_title(title: Title) -> str:
  return title.title


class Character(NamedTuple):
  """Character class."""

  # Name of the character.
  name: str

  # A single sentence describing the character.
  description: str

  @classmethod
  def from_string(cls, text: str):
    elements = text.split(DESCRIPTION_MARKER)
    if len(elements) == 2:
      name = elements[0].strip()
      description = elements[1].strip()
      return cls(name, description)
    else:
      return None


def get_character_description(character: Character) -> str:
  return character.description


class Characters(NamedTuple):
  """Characters class, containing main characters and their descriptions."""

  # A dictionary of character descriptions.
  character_descriptions: Dict[str, str]

  @classmethod
  def from_string(cls, text: str):
    """Parses the characters from the generated text."""
    text = text.strip()

    # Extracts the character descriptions.
    character_descriptions = {}
    elements = extract_elements(text, CHARACTER_MARKER, STOP_MARKER)
    for text_character in elements:
      character = Character.from_string(text_character)
      if character is not None:
        character_descriptions[character.name] = character.description
    return cls(character_descriptions)

  def to_string(self):
    s = '\n'
    for name, description in self.character_descriptions.items():
      s += '\n' + CHARACTER_MARKER + ' ' + name + ' ' + DESCRIPTION_MARKER + ' '
      s += description + ' ' + STOP_MARKER + '\n'
    s += END_MARKER
    return s


def get_character_descriptions(characters: Characters) -> Dict[str, str]:
  return characters.character_descriptions


class Scene(NamedTuple):
  """Scene class."""

  # The name of the place where the scene unfolds.
  place: str

  # Name of the plot element (e.g., Beginning, Middle, Conclusion).
  plot_element: str

  # A short description of action/story/dramatic event occuring in the scene.
  beat: str

  def to_string(self):
    s = PLACE_ELEMENT + ' ' + self.place + '\n'
    s += PLOT_ELEMENT + ' ' + self.plot_element + '\n'
    s += BEAT_ELEMENT + ' ' + self.beat + '\n'
    return s


class Scenes(NamedTuple):
  """Scenes class."""

  # A list of scenes, with place, characters, plot element and beat.
  scenes: List[Scene]

  @classmethod
  def from_string(cls, text: str):
    """Parse scenes from generated scenes_text."""

    places = extract_elements(text, PLACE_ELEMENT, PLOT_ELEMENT)
    plot_elements = extract_elements(text, PLOT_ELEMENT, BEAT_ELEMENT)
    beats = extract_elements(text, BEAT_ELEMENT, '\n')

    # Get the number of complete scenes.
    num_complete_scenes = min([len(places), len(plot_elements), len(beats)])
    scenes = []
    for i in range(num_complete_scenes):
      scenes.append(
          Scene(Place.format_name(places[i]), plot_elements[i], beats[i]))
    scenes = cls(scenes)
    return scenes

  def to_string(self):
    s = ''
    for scene in self.scenes:
      s += '\n' + scene.to_string()
    s += END_MARKER
    return s

  def num_places(self):
    return len(set([scene.place for scene in self.scenes]))

  def num_scenes(self) -> int:
    return len(self.scenes)


class Place(NamedTuple):
  """Place class."""

  # Place name.
  name: str

  # Place description.
  description: str

  @classmethod
  def format_name(cls, name: str):
    if name.find('.') == -1:
      name = name + '.'
    return name

  @classmethod
  def from_string(cls, place_name: str, place_text: str):
    place_text += END_MARKER
    description = extract_elements(place_text, DESCRIPTION_ELEMENT, END_MARKER)
    return cls(place_name, description[0])

  @classmethod
  def format_prefix(cls, name):
    s = PLACE_ELEMENT + name + '\n' + DESCRIPTION_ELEMENT
    return s

  def to_string(self):
    s = self.format_prefix(self.name) + self.description + '\n\n'
    return s


def get_place_description(place: Place):
  return place.description


class Story(NamedTuple):
  """Story class."""

  # A storyline is a single sentence summary of the whole plot.
  storyline: str

  # A title for the story.
  title: str

  # Map from character names to full descriptions.
  character_descriptions: Dict[str, str]

  # Map from place names to full descriptions.
  place_descriptions: Dict[str, Place]

  # List of scenes.
  scenes: Scenes

  # List of dialogs, one for each scene.
  dialogs: List[str]


def extract_elements(text: str, begin: str, end: str) -> List[str]:
  """Extracts elements from a text string given string and ending markers."""

  results = []
  start = 0
  while True:
    start = text.find(begin, start)
    if start == -1:
      return results
    finish = text.find(end, start)
    if finish == -1:
      return results
    results.append(text[start + len(begin):finish].strip())
    start = finish + len(end)


def strip_remove_end(text: str) -> str:
  text = text.strip()
  end_marker_stripped = END_MARKER.strip()
  if text.endswith(end_marker_stripped):
    text = text[:-len(end_marker_stripped)]
  return text


# ------------------------------------------------------------------------------
# Rendering of generated stories
# ------------------------------------------------------------------------------


def render_story(story: Story) -> str:
  """Render the story in fountain format."""

  lines = []
  lines.append(f'Title: {story.title}')
  lines.append('Author: Co-written by ________ and Dramatron')
  lines.append(
      'Dramatron was developed by Piotr Mirowski and Kory W. Mathewson, '
      'with additional contributions by Juliette Love and Jaylen Pittman, '
      'and is based on a prototype by Richard Evans.')
  lines.append('Dramatron relies on user-provided language models.')
  lines.append('')
  lines.append('====')
  lines.append('')

  lines.append(f'The script is based on the storyline:\n{story.storyline}')
  lines.append('')
  if story.character_descriptions is not None:
    for name, description in story.character_descriptions.items():
      lines.append(f'{name}: {description}')
      lines.append('')

  # For each scene, render scene information.
  if story.scenes is not None:
    scenes = story.scenes.scenes
    for i, scene in enumerate(scenes):
      lines.append(f'Scene {i+1}')
      lines.append(f'{PLACE_ELEMENT}{scene.place}')
      lines.append(f'{PLOT_ELEMENT}{scene.plot_element}')
      lines.append(f'{BEAT_ELEMENT}{scene.beat}')
      lines.append('')
  else:
    scenes = []

  lines.append('====')
  lines.append('')

  # For each scene, render the scene's place description, characters and dialog.
  for i, scene in enumerate(scenes):

    # Output the places and place descriptions.
    lines.append(f'INT/EXT. {scene.place} - Scene {i+1}')
    place_descriptions = story.place_descriptions
    if (not place_appears_earlier(scene.place, story, i) and
        place_descriptions is not None and scene.place in place_descriptions):
      lines.append('')
      lines.append(get_place_description(place_descriptions[scene.place]))

    # Output the characters and descriptions.
    lines.append('')
    for c in story.character_descriptions.keys():
      if c in scene.beat and not character_appears_earlier(c, story, i):
        lines.append(story.character_descriptions[c])

    # Output the dialog.
    if story.dialogs is not None and len(story.dialogs) > i:
      lines.append('')
      lines_dialog = strip_remove_end(str(story.dialogs[i]))
      lines.append(lines_dialog)
      lines.append('')
      lines.append('')

  return '\n'.join(lines)


def place_appears_earlier(place: str, story: Story, index: int) -> bool:
  """Return True if the place appears earlier in the story."""

  for i in range(index):
    scene = story.scenes.scenes[i]
    if scene.place == place:
      return True
  return False


def character_appears_earlier(character: str, story: Story, index: int) -> bool:
  """Return True if the character appears earlier in the story."""

  for i in range(index):
    scene = story.scenes.scenes[i]
    if character in scene.beat:
      return True
  return False


def render_prompts(prompts):
  """Render the prompts."""

  def _format_prompt(prompt, name):
    prompt_str = '=' * 80 + '\n'
    prompt_str += 'PROMPT (' + name + ')\n'
    prompt_str += '=' * 80 + '\n\n'
    prompt_str += str(prompt) + '\n\n'
    return prompt_str

  prompts_str = _format_prompt(prompts['title'], 'title')
  prompts_str += _format_prompt(prompts['characters'], 'characters')
  prompts_str += _format_prompt(prompts['scenes'], 'scenes')
  places = prompts['places']
  if places is not None:
    for k, prompt in enumerate(places):
      prompts_str += _format_prompt(prompt, 'place ' + str(k + 1))
  dialogs = prompts['dialogs']
  if dialogs is not None:
    for k, prompt in enumerate(dialogs):
      prompts_str += _format_prompt(prompt, 'dialog ' + str(k + 1))
  return prompts_str


# ------------------------------------------------------------------------------
# Language API definition
# ------------------------------------------------------------------------------

_MAX_RETRIES = 10
_TIMEOUT = 120.0


class LanguageResponse(NamedTuple):
  prompt: str
  prompt_length: int
  text: str
  text_length: int


class LanguageAPI:
  """Language model wrapper."""

  def __init__(self,
               sample_length: int,
               model: Optional[str] = None,
               model_param: Optional[str] = None,
               config_sampling: Optional[dict] = None,
               seed: Optional[int] = None,
               max_retries: int = _MAX_RETRIES,
               timeout: float = _TIMEOUT):
    """Initializer.

    Args:
      sample_length: Length of text to sample from model.
      model: The model name to correct to. An error will be raised if it does
        not exist.
      model_param: Model parameter.
      config_sampling: Sampleing parameters.
      seed: Random seed for sampling.
      max_retries: Maximum number of retries for the remote API.
      timeout: Maximum waiting timeout
    """
    self._sample_length = sample_length
    self._model = model
    self._model_param = model_param
    self._config_sampling = config_sampling
    self._seed = seed
    self._max_retries = max_retries
    self._timeout = timeout

  @property
  def default_sample_length(self):
    return self._sample_length

  @property
  def model(self):
    return self._model

  @property
  def model_param(self):
    return self._model_param

  @property
  def model_metadata(self):
    return None

  @property
  def seed(self):
    return self._seed

  @property
  def config_sampling(self):
    return self._config_sampling

  def sample(self,
             prompt: str,
             sample_length: Optional[int] = None,
             seed: Optional[int] = None,
             num_samples: int = 1):
    """Sample model with provided prompt, optional sample_length and seed."""
    raise NotImplementedError('sample method not implemented in generic class')


class FilterAPI:
  """Filter model wrapper."""

  def validate(self, text: str):
    raise NotImplementedError('validate not implemented in generic class')


# ------------------------------------------------------------------------------
# Dramatron Generator
# ------------------------------------------------------------------------------


def generate_text(generation_prompt: str,
                  client: LanguageAPI,
                  filter: Optional[FilterAPI] = None,
                  sample_length: Optional[int] = None,
                  max_paragraph_length: int = MAX_PARAGRAPH_LENGTH,
                  seed: Optional[int] = None,
                  num_samples: int = 1,
                  max_num_repetitions: Optional[int] = None) -> str:
  """Generate text using the generation prompt."""

  # To prevent lengthy generation loops, we cap the number of calls to the API.
  if sample_length is None:
    sample_length = client.default_sample_length
  max_num_calls = int(max_paragraph_length / sample_length) + 1
  num_calls = 0

  result = ''
  while True:
    prompt = generation_prompt + result
    success, current_seed = False, seed
    while success is False:
      t0 = time.time()
      responses = client.sample(
          prompt=prompt,
          sample_length=sample_length,
          seed=current_seed,
          num_samples=num_samples)
      t1 = time.time()
      # Get the first result from the list of responses
      response = responses[0]
      if filter is not None and not filter.validate(response.text):
        return 'Content was filtered out.' + END_MARKER
      if max_num_repetitions:
        success = not detect_loop(
            response.text, max_num_repetitions=max_num_repetitions)
        if not success:
          current_seed += 1
          if current_seed > (seed + MAX_NUM_ATTEMPTS_GET_OUT_OF_LOOP):
            success = True
          else:
            continue
      else:
        success = True

    result = result + response.text
    num_calls += 1

    # Attempt to find the END_MARKER
    index = result.find(END_MARKER)
    if index != -1:
      return result[:index] + END_MARKER

    # Attempt to find the start of a new example
    index = result.find('Example ')
    if index != -1:
      return result[:index] + END_MARKER

    if max_paragraph_length is not None and len(result) > max_paragraph_length:
      return result + END_MARKER
    if num_calls >= max_num_calls:
      return result + END_MARKER

  return result


def generate_text_no_loop(generation_prompt: str,
                          client: LanguageAPI,
                          filter: Optional[FilterAPI] = None,
                          sample_length: Optional[int] = None,
                          max_paragraph_length: int = MAX_PARAGRAPH_LENGTH,
                          seed: Optional[int] = None,
                          num_samples: int = 1) -> str:
  """Generate text using the generation prompt, without any loop."""
  return generate_text(
      generation_prompt=generation_prompt,
      client=client,
      filter=filter,
      sample_length=sample_length,
      max_paragraph_length=sample_length,
      seed=seed,
      max_num_repetitions=None,
      num_samples=num_samples)


def generate_title(storyline: str,
                   prefixes: Dict[str, str],
                   client: LanguageAPI,
                   filter: Optional[FilterAPI] = None,
                   seed: Optional[int] = None,
                   num_samples: int = 1):
  """Generate a title given a storyline, and client."""

  # Combine the prompt and storyline as a helpful generation prefix
  titles_prefix = prefixes['TITLES_PROMPT'] + storyline + ' ' + TITLE_ELEMENT
  title_text = generate_text_no_loop(
      generation_prompt=titles_prefix,
      client=client,
      filter=filter,
      sample_length=SAMPLE_LENGTH_TITLE,
      seed=seed,
      num_samples=num_samples)
  title = Title.from_string(TITLE_ELEMENT + title_text)
  return (title, titles_prefix)


def generate_characters(
    storyline: str,
    prefixes: Dict[str, str],
    client: LanguageAPI,
    filter: Optional[FilterAPI] = None,
    seed: Optional[int] = None,
    max_paragraph_length: int = (MAX_PARAGRAPH_LENGTH_CHARACTERS),
    num_samples: int = 1):
  """Generate characters given a storyline, prompt, and client."""

  # Combine the prompt and storyline as a helpful generation prefix
  characters_prefix = prefixes['CHARACTERS_PROMPT'] + storyline
  characters_text = generate_text(
      generation_prompt=characters_prefix,
      client=client,
      filter=filter,
      seed=seed,
      max_paragraph_length=max_paragraph_length,
      num_samples=num_samples)
  characters = Characters.from_string(characters_text)

  return (characters, characters_prefix)


def generate_scenes(storyline: str,
                    character_descriptions: Dict[str, str],
                    prefixes: Dict[str, str],
                    client: LanguageAPI,
                    filter: Optional[FilterAPI] = None,
                    seed: Optional[int] = None,
                    max_paragraph_length: int = (MAX_PARAGRAPH_LENGTH_SCENES),
                    num_samples: int = 1):
  """Generate scenes given storyline, prompt, main characters, and client."""

  scenes_prefix = prefixes['SCENE_PROMPT'] + storyline + '\n'
  for name in character_descriptions:
    scenes_prefix += character_descriptions[name] + '\n'
  scenes_prefix += '\n' + SCENES_MARKER
  scenes_text = generate_text(
      generation_prompt=scenes_prefix,
      client=client,
      filter=filter,
      seed=seed,
      max_paragraph_length=max_paragraph_length,
      num_samples=num_samples)
  scenes = Scenes.from_string(scenes_text)

  return (scenes, scenes_prefix)


def generate_place_descriptions(storyline: str,
                                scenes: Scenes,
                                prefixes: Dict[str, str],
                                client: LanguageAPI,
                                filter: Optional[FilterAPI] = None,
                                seed: Optional[int] = None,
                                num_samples: int = 1):
  """Generate a place description given a scene object and a client."""

  place_descriptions = {}

  # Get unique place names from the scenes.
  unique_place_names = set([scene.place for scene in scenes.scenes])

  # Build a unique place prefix prompt.
  place_prefix = prefixes['SETTING_PROMPT'] + storyline + '\n'

  # Build a list of place descriptions for each place
  place_prefixes = []
  for place_name in unique_place_names:
    place_suffix = Place.format_prefix(place_name)
    place_text = generate_text(
        generation_prompt=place_prefix + place_suffix,
        client=client,
        filter=filter,
        sample_length=SAMPLE_LENGTH_PLACE,
        seed=seed,
        num_samples=num_samples)
    place_text = place_suffix + place_text
    place_descriptions[place_name] = Place.from_string(place_name, place_text)
    place_prefixes.append(place_prefix + place_suffix)

  return (place_descriptions, place_prefixes)


def prefix_summary(storyline: str,
                   scenes: List[Scene],
                   concatenate_scenes_in_summary: bool = False) -> str:
  """Assemble the summary part of the dialog prefix."""

  summary = SUMMARY_ELEMENT + storyline + '\n'
  if len(scenes) > 1:
    summary += PREVIOUS_ELEMENT + scenes[len(scenes) - 2].beat + '\n'
  return summary


def detect_loop(text: str, max_num_repetitions: int = MAX_NUM_REPETITIONS):
  """Detect loops in generated text."""

  blocks = text.split('\n\n')
  num_unique_blocks = collections.Counter(blocks)
  for block in blocks:
    num_repetitions = num_unique_blocks[block]
    if num_repetitions > max_num_repetitions:
      print(f'Detected {num_repetitions} repetitions of block:\n{block}')
      return True
  return False


def generate_dialog(storyline: str,
                    scenes: List[Scene],
                    character_descriptions: Dict[str, str],
                    place_descriptions: Dict[str, Place],
                    prefixes: Dict[str, str],
                    max_paragraph_length: int,
                    client: LanguageAPI,
                    filter: Optional[FilterAPI] = None,
                    max_num_repetitions: Optional[int] = None,
                    seed: Optional[int] = None,
                    num_samples: int = 1):
  """Generate dialog given a scene object and a client."""

  scene = scenes[-1]

  place_t = PLACE_ELEMENT + scene.place + '\n'
  if scene.place in place_descriptions:
    place_description = place_descriptions[scene.place]
    if place_description:
      place_t += DESCRIPTION_ELEMENT + place_description.description
      place_t += '\n'

  # Build the characters information for the scene
  characters_t = ''
  if character_descriptions:
    characters_t += CHARACTERS_ELEMENT
    for name in character_descriptions:
      if name in scene.beat:
        characters_t += character_descriptions[name] + '\n'

  plot_element_t = PLOT_ELEMENT + scene.plot_element + '\n'

  summary_t = prefix_summary(
      storyline, scenes, concatenate_scenes_in_summary=False)

  beat_t = BEAT_ELEMENT + scene.beat + '\n'

  dialog_prefix = (
      prefixes['DIALOG_PROMPT'] + place_t + characters_t + plot_element_t +
      summary_t + beat_t)
  dialog_prefix += '\n' + DIALOG_MARKER + '\n'

  dialog = generate_text(
      generation_prompt=dialog_prefix,
      client=client,
      filter=filter,
      seed=seed,
      max_paragraph_length=max_paragraph_length,
      max_num_repetitions=max_num_repetitions,
      num_samples=num_samples)

  return (dialog, dialog_prefix)


def diff_prompt_change_str(prompt_before: str, prompt_after: str) -> str:
  """Return a text diff on prompt sets `prompt_before` and `prompt_after`."""

  # For the current element, compare prompts line by line.
  res = difflib.unified_diff(
      prompt_before.split('\n'), prompt_after.split('\n'))
  diff = ''
  for line in res:
    line = line.strip()
    if line != '---' and line != '+++' and not line.startswith('@@'):
      if len(line) > 1 and (line.startswith('+') or line.startswith('-')):
        diff += line + '\n'
  if diff.endswith('\n'):
    diff = diff[:-1]
  return diff


def diff_prompt_change_list(prompt_before: List[str],
                            prompt_after: List[str]) -> str:
  """Return a text diff on prompt sets `prompt_before` and `prompt_after`."""

  # Handle deletions and insertions.
  len_before = len(prompt_before)
  len_after = len(prompt_after)
  if len_before > len_after:
    return 'Deleted element'
  if len_before < len_after:
    return 'Added new element'

  diffs = [
      diff_prompt_change_str(a, b)
      for (a, b) in zip(prompt_before, prompt_after)
  ]
  return '\n'.join([diff for diff in diffs if len(diff) > 0])


def diff_prompt_change_scenes(prompt_before: List[Scene],
                              prompt_after: List[Scene]) -> str:
  """Return a text diff on prompt sets `prompt_before` and `prompt_after`."""

  # Handle deletions and insertions.
  len_before = len(prompt_before)
  len_after = len(prompt_after)
  if len_before > len_after:
    return 'Deleted element'
  if len_before < len_after:
    return 'Added new element'

  diffs = [
      diff_prompt_change_list([a.place, a.plot_element, a.beat],
                              [b.place, b.plot_element, b.beat])
      for (a, b) in zip(prompt_before, prompt_after)
  ]
  return '\n'.join([diff for diff in diffs if len(diff) > 0])


def diff_prompt_change_dict(prompt_before: Dict[str, str],
                            prompt_after: Dict[str, str]) -> str:
  """Return a text diff on prompt sets `prompt_before` and `prompt_after`."""

  # Loop over the keys in the prompts to compare them one by one.
  keys_before = sorted(prompt_before.keys())
  keys_after = sorted(prompt_after.keys())
  diffs = [
      diff_prompt_change_str(a, b) for (a, b) in zip(keys_before, keys_after)
  ]
  diff_keys = '\n'.join([diff for diff in diffs if len(diff) > 0])
  # Loop over the values in the prompts to compare them one by one.
  values_before = sorted(prompt_before.values())
  values_after = sorted(prompt_after.values())
  diffs = [
      diff_prompt_change_str(a, b)
      for (a, b) in zip(values_before, values_after)
  ]
  diff_values = '\n'.join([diff for diff in diffs if len(diff) > 0])
  return diff_keys + diff_values


class StoryGenerator:
  """Generate a story from the provided storyline, using the client provided."""

  level_names = ('storyline', 'title', 'characters', 'scenes', 'places',
                 'dialogs')

  def __init__(
      self,
      storyline: str,
      prefixes: Dict[str, str],
      max_paragraph_length: int = 1024,
      max_paragraph_length_characters: int = (MAX_PARAGRAPH_LENGTH_CHARACTERS),
      max_paragraph_length_scenes: int = (MAX_PARAGRAPH_LENGTH_SCENES),
      num_samples: int = 1,
      client: Optional[LanguageAPI] = None,
      filter: Optional[FilterAPI] = None):
    self._prefixes = prefixes
    self._max_paragraph_length = max_paragraph_length
    self._max_paragraph_length_characters = max_paragraph_length_characters
    self._max_paragraph_length_scenes = max_paragraph_length_scenes
    self._num_samples = num_samples
    self._client = client
    self._filter = filter

    # Prompts and outputs of the hierarchical generator are organised in levels.
    self.prompts = {
        'title': '',
        'characters': '',
        'scenes': '',
        'places': {
            '': ''
        },
        'dialogs': ['']
    }
    self._title = Title('')
    self._characters = Characters({'': ''})
    self._scenes = Scenes([Scene('', '', '')])
    self._places = {'': Place('', '')}
    self._dialogs = ['']

    # History of interventions.
    self.interventions = {}
    self._set_storyline(storyline)

  def _set_storyline(self, storyline: str):
    """Set storyline and initialise the outputs of the generator."""
    self._level = 0

    # Add period to the end of the storyline, unless there is already one there.
    if storyline.find('.') == -1:
      storyline = storyline + '.'
    self._storyline = storyline

    # Keep track of each storyline intervention.
    timestamp = time.time()
    self.interventions[timestamp] = 'STORYLINE\n' + storyline

  @property
  def seed(self):
    return self._client.seed

  @property
  def title(self) -> Title:
    """Return the title."""
    return self._title

  @property
  def characters(self) -> Characters:
    """Return the characters."""
    return self._characters

  @property
  def scenes(self) -> Scenes:
    """Return the title."""
    return self._scenes

  @property
  def places(self) -> Dict[str, Place]:
    """Return the places."""
    return self._places

  @property
  def dialogs(self) -> List[str]:
    """Return the dialogs."""
    return self._dialogs

  def title_str(self) -> str:
    """Return the title as a string."""
    return self._title.title

  def num_scenes(self) -> int:
    """Return the number of scenes."""
    return self._scenes.num_scenes()

  def step(self,
           level: Optional[int] = None,
           seed: Optional[int] = None,
           idx: Optional[int] = None) -> bool:
    """Step down a level in the hierarchical generation of a story."""

    # Move to the next level of hierarchical generation.
    if level is None:
      level = self._level
    if level < 0 or level >= len(self.level_names):
      raise ValueError('Invalid level encountered on step.')
    level += 1
    self._level = level

    # Keep track of each step intervention.
    timestamp = time.time()
    self.interventions[timestamp] = 'STEP ' + str(level) + '\n'

    if level == 1:
      # Step 1: Generate title given a storyline.
      (title, titles_prefix) = generate_title(
          storyline=self._storyline,
          prefixes=self._prefixes,
          client=self._client,
          filter=self._filter,
          num_samples=self._num_samples,
          seed=seed)
      self._title = title
      self.prompts['title'] = titles_prefix
      self.interventions[timestamp] += title.to_string()
      success = len(title.title) > 0
      return success

    if level == 2:
      # Step 2: Generate characters given a storyline.
      (characters, character_prompts) = generate_characters(
          storyline=self._storyline,
          prefixes=self._prefixes,
          client=self._client,
          filter=self._filter,
          num_samples=self._num_samples,
          max_paragraph_length=self._max_paragraph_length_characters,
          seed=seed)
      self._characters = characters
      self.prompts['characters'] = character_prompts
      self.interventions[timestamp] += characters.to_string()
      success = len(characters.character_descriptions) > 0
      return success

    if level == 3:
      # Step 3: Generate sequence of scenes given a storyline and characters.
      characters = self._characters
      (scenes, scene_prompts) = generate_scenes(
          storyline=self._storyline,
          character_descriptions=get_character_descriptions(characters),
          prefixes=self._prefixes,
          client=self._client,
          filter=self._filter,
          num_samples=self._num_samples,
          max_paragraph_length=self._max_paragraph_length_scenes,
          seed=seed)
      self._scenes = scenes
      self.prompts['scenes'] = scene_prompts
      self.interventions[timestamp] += scenes.to_string()
      success = len(scenes.scenes) > 0
      return success

    if level == 4:
      # Step 4: For each scene, generate place descriptions given place name.
      scenes = self._scenes
      (place_descriptions, place_prompts) = generate_place_descriptions(
          storyline=self._storyline,
          scenes=scenes,
          prefixes=self._prefixes,
          client=self._client,
          filter=self._filter,
          num_samples=self._num_samples,
          seed=seed)
      self._places = place_descriptions
      self.prompts['places'] = place_prompts
      for place_name in place_descriptions:
        place = place_descriptions[place_name]
        if place:
          self.interventions[timestamp] += place.to_string()
      num_places = scenes.num_places()
      success = (len(place_descriptions) == num_places) and num_places > 0
      return success

    if level == 5:
      # Step 5: For each scene, generate dialog from scene information.
      title = self._title
      characters = self._characters
      scenes = self._scenes
      place_descriptions = self._places
      if idx is None:
        (dialogs, dialog_prompts) = zip(*[
            generate_dialog(
                storyline=self._storyline,
                scenes=scenes.scenes[:(k + 1)],
                character_descriptions=(characters.character_descriptions),
                place_descriptions=place_descriptions,
                prefixes=self._prefixes,
                max_paragraph_length=self._max_paragraph_length,
                max_num_repetitions=MAX_NUM_REPETITIONS,
                client=self._client,
                filter=self._filter,
                num_samples=self._num_samples,
                seed=seed) for k in range(len(scenes.scenes))
        ])
      else:
        num_scenes = self._scenes.num_scenes()
        while len(self._dialogs) < num_scenes:
          self._dialogs.append('')
        while len(self.prompts['dialogs']) < num_scenes:
          self.prompts['dialogs'].append('')
        if idx >= num_scenes or idx < 0:
          raise ValueError('Invalid scene index.')
        dialogs = self._dialogs
        dialog_prompts = self.prompts['dialogs']
        dialogs[idx], dialog_prompts[idx] = generate_dialog(
            storyline=self._storyline,
            scenes=scenes.scenes[:(idx + 1)],
            character_descriptions=(characters.character_descriptions),
            place_descriptions=place_descriptions,
            prefixes=self._prefixes,
            max_paragraph_length=self._max_paragraph_length,
            max_num_repetitions=MAX_NUM_REPETITIONS,
            client=self._client,
            filter=self._filter,
            num_samples=self._num_samples,
            seed=seed)
      self._dialogs = dialogs
      self.prompts['dialogs'] = dialog_prompts
      for dialog in dialogs:
        self.interventions[timestamp] += str(dialog)
      return True

  def get_story(self):
    if self._characters is not None:
      character_descriptions = get_character_descriptions(self._characters)
    else:
      character_descriptions = None
    return Story(
        storyline=self._storyline,
        title=self._title.title,
        character_descriptions=character_descriptions,
        place_descriptions=self._places,
        scenes=self._scenes,
        dialogs=self._dialogs)

  def rewrite(self, text, level=0, entity=None):
    if level < 0 or level >= len(self.level_names):
      raise ValueError('Invalid level encountered on step.')
    prompt_diff = None

    if level == 0:
      # Step 0: Rewrite the storyline and begin new story.
      prompt_diff = diff_prompt_change_str(self._storyline, text)
      self._set_storyline(text)

    if level == 1:
      # Step 1: Rewrite the title.
      title = Title.from_string(text)
      prompt_diff = diff_prompt_change_str(self._title.title, title.title)
      self._title = title

    if level == 2:
      # Step 2: Rewrite the characters.
      characters = Characters.from_string(text)
      prompt_diff = diff_prompt_change_dict(
          self._characters.character_descriptions,
          characters.character_descriptions)
      self._characters = characters

    if level == 3:
      # Step 3: Rewrite the sequence of scenes.
      scenes = Scenes.from_string(text)
      prompt_diff = diff_prompt_change_scenes(self._scenes.scenes,
                                              scenes.scenes)
      self._scenes = scenes

    if level == 4:
      # Step 4: For a given place, rewrite its place description.
      place_descriptions = self._places
      if entity in place_descriptions:
        place_prefix = Place.format_prefix(entity)
        text = place_prefix + text
        place = Place.from_string(entity, text)
        prompt_diff = diff_prompt_change_str(self._places[entity].name,
                                             place.name)
        prompt_diff += '\n' + diff_prompt_change_str(
            self._places[entity].description, place.description)

        self._places[entity] = place

    if level == 5:
      # Step 5: Rewrite the dialog of a given scene.
      dialogs = self._dialogs
      num_scenes = len(self._scenes.scenes)
      if entity >= 0 and entity < num_scenes:
        prompt_diff = diff_prompt_change_str(self._dialogs[entity], text)
        self._dialogs[entity] = text

    # Keep track of each rewrite intervention.
    if prompt_diff is not None and len(prompt_diff) > 0:
      timestamp = time.time()
      self.interventions[timestamp] = 'REWRITE ' + self.level_names[level]
      if entity:
        self.interventions[timestamp] += ' ' + str(entity)
      self.interventions[timestamp] += prompt_diff

  def complete(self,
               level=0,
               seed=None,
               entity=None,
               sample_length=SAMPLE_LENGTH):
    if level < 0 or level >= len(self.level_names):
      raise ValueError('Invalid level encountered on step.')
    prompt_diff = None

    if level == 2:
      # Step 2: Complete the characters.
      text_characters = self._characters.to_string()
      text_characters = strip_remove_end(text_characters)
      prompt = self.prompts['characters'] + text_characters
      text = generate_text(
          generation_prompt=prompt,
          client=self._client,
          filter=self._filter,
          sample_length=sample_length,
          max_paragraph_length=sample_length,
          seed=seed,
          num_samples=1)
      new_characters = Characters.from_string(text_characters + text)
      prompt_diff = diff_prompt_change_dict(
          self._characters.character_descriptions,
          new_characters.character_descriptions)
      self._characters = new_characters

    if level == 3:
      # Step 3: Complete the sequence of scenes.
      text_scenes = self._scenes.to_string()
      text_scenes = strip_remove_end(text_scenes)
      prompt = self.prompts['scenes'] + text_scenes
      text = generate_text(
          generation_prompt=prompt,
          client=self._client,
          filter=self._filter,
          sample_length=sample_length,
          max_paragraph_length=sample_length,
          seed=seed,
          num_samples=1)
      new_scenes = Scenes.from_string(text_scenes + text)
      prompt_diff = diff_prompt_change_scenes(self._scenes.scenes,
                                              new_scenes.scenes)
      self._scenes = new_scenes

    if level == 5:
      # Step 5: Complete the dialog of a given scene.
      dialogs = self._dialogs
      num_scenes = len(self._scenes.scenes)
      while len(self._dialogs) < num_scenes:
        self._dialogs.append('')
      while len(self.prompts['dialogs']) < num_scenes:
        self.prompts['dialogs'].append('')
      if entity >= 0 and entity < num_scenes:
        prompt = (self.prompts['dialogs'][entity] + self._dialogs[entity])
        text = generate_text(
            generation_prompt=prompt,
            client=self._client,
            filter=self._filter,
            sample_length=sample_length,
            max_paragraph_length=sample_length,
            seed=seed,
            num_samples=1)
        new_dialog = self._dialogs[entity] + text
        prompt_diff = diff_prompt_change_str(self._dialogs[entity], new_dialog)
        self._dialogs[entity] = new_dialog

    # Keep track of each rewrite intervention.
    if prompt_diff is not None and len(prompt_diff) > 0:
      timestamp = time.time()
      self.interventions[timestamp] = 'COMPLETE ' + self.level_names[level]
      if entity:
        self.interventions[timestamp] += ' ' + str(entity)
      self.interventions[timestamp] += prompt_diff


# ------------------------------------------------------------------------------
# UI
# ------------------------------------------------------------------------------


class GenerationAction:
  NEW = 1
  CONTINUE = 2
  REWRITE = 3


class GenerationHistory:
  """Custom data structure to handle the history of GenerationAction edits:

  NEW, CONTINUE or REWRITE. Consecutive REWRITE edits do not add to history.
  """

  def __init__(self):
    self._items = []
    self._actions = []
    self._idx = -1
    self._locked = False

  def _plain_add(self, item, action: GenerationAction):
    self._items.append(item)
    self._actions.append(action)
    self._idx = len(self._items) - 1
    return self._idx

  def add(self, item, action: GenerationAction):
    if len(self._items) == 0 or action != GenerationAction.REWRITE:
      return self._plain_add(item, action)
    last_action = self._actions[-1]
    if last_action != GenerationAction.REWRITE:
      return self._plain_add(item, action)
    self._items[self._idx] = item
    return self._idx

  def previous(self):
    if len(self._items) == 0:
      return None
    self._idx = max(self._idx - 1, 0)
    return self._items[self._idx]

  def next(self):
    if len(self._items) == 0:
      return None
    self._idx = min(self._idx + 1, len(self._items) - 1)
    return self._items[self._idx]

filter = None

print('Dramatron set-up complete.')

In [ ]:
#@title Custom Language API

#@markdown This cell should contain code with your language API.

class CustomLanguageAPI(LanguageAPI):
  """A class wrapping the language model API."""

  def __init__(self,
               sample_length: int,
               model: Optional[str] = None,
               model_param: Optional[str] = None,
               config_sampling: Optional[dict] = None,
               seed: Optional[int] = None,
               max_retries: int = _MAX_RETRIES,
               timeout: float = _TIMEOUT):
    """Initializer.

    Args:
      sample_length: Length of text to sample from model.
      model: The model name to correct to. An error will be raised if it does
        not exist.
      model_param: Custom language model params.
      config_sampling: ConfigDict with parameters.
      seed: Random seed for sampling.
      max_retries: Maximum number of retries for the remote API.
      timeout: Maximum waiting timeout
    """
    raise NotImplementedError('init method not implemented')

  @property
  def client(self):
    return self._client

  @property
  def model_metadata(self):
    return {'engine': self._model,
            'model_param': self._model_param,
            'max_tokens': self._sample_length}

  def sample(self,
             prompt: str,
             sample_length: Optional[int] = None,
             seed: Optional[int] = None,
             num_samples: int = 1):
    raise NotImplementedError('sample method not implemented')


# Create the config.
config = {}
config['language_api_name'] = None
config['model_param'] = None
config['model_name'] = None
config['max_retries'] = MAX_RETRIES
config['sample_length'] = SAMPLE_LENGTH
config['max_paragraph_length'] = MAX_PARAGRAPH_LENGTH
config['max_paragraph_length_characters'] = MAX_PARAGRAPH_LENGTH_CHARACTERS
config['max_paragraph_length_scenes'] = MAX_PARAGRAPH_LENGTH_SCENES
config['sampling'] = {}
config['sampling']['prob'] = SAMPLING_PROB
config['sampling']['temp'] = SAMPLING_TEMP
config['sampling']['frequency_penalty'] = 0.23
config['sampling']['presence_penalty'] = 0.23
config['prefixes'] = {}
config['file_dir'] = None

print('Config:')
for key, value in config.items():
  if key != 'prefixes':
    print(f'{key}: {value}')

try:
  client = CustomLanguageAPI(
      model_param=config['model_param'],
      model=config['model_name'],
      seed=DEFAULT_SEED,
      sample_length=config['sample_length'],
      max_retries=config['max_retries'],
      config_sampling=config['sampling'])

  print(f'Client model metadata: {client.model_metadata}')
except:
  print('CustomLanguageAPI not implemented yet')

In [ ]:
#@title (Optional) Perspective API { run: "auto" }

PERSPECTIVE_API_URL = 'https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze'

#@markdown The language model may generate offensive text. If you choose, you can provide a [Perspective API](https://perspectiveapi.com/) key, which will hide model outputs that exceed a toxicity threshold and prompt you to regenerate the text.
PERSPECTIVE_API_KEY = '' #@param {type:"string"}

#@markdown Perspective API toxicity thresholds across different attributes (default value: 0.8).
PERSPECTIVE_API_TOXICITY = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
PERSPECTIVE_API_SEVERE_TOXICITY = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
PERSPECTIVE_API_IDENTITY_ATTACK = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
PERSPECTIVE_API_INSULT = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
PERSPECTIVE_API_SEXUALLY_EXPLICIT = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}

PERSPECTIVE_API_THRESHOLDS = {
    "TOXICITY": PERSPECTIVE_API_TOXICITY,
    "SEVERE_TOXICITY": PERSPECTIVE_API_SEVERE_TOXICITY,
    "IDENTITY_ATTACK": PERSPECTIVE_API_IDENTITY_ATTACK,
    "INSULT": PERSPECTIVE_API_INSULT,
    "SEXUALLY_EXPLICIT": PERSPECTIVE_API_SEXUALLY_EXPLICIT
}


class PerspectiveAPI(FilterAPI):
  """Wraps the Perspective API model."""

  def __init__(self,
               key: str,
               thresholds: dict):
    """Initializer:

    Args:
      key: Perspective API key.
      thresholds: Thresholds for attributes.
    """
    self._key = key
    self._thresholds = thresholds

  def get_scores(self, text: str):
    """Get the scores from the Perspective API comment analyzer for text."""

    input_data = {
        'comment': {'text': text},
        'languages': ['en'],
        'requestedAttributes': {attribute:{} for attribute in self._thresholds}
    }
    response = requests.post(
        PERSPECTIVE_API_URL,
        params={'key': self._key},
        headers={'Content-Type': 'application/json'},
        data=json.dumps(input_data)
    )
    output_data = response.json()
    scores = {}
    for attribute in self._thresholds:
      score = output_data['attributeScores'][attribute]['summaryScore']['value']
      scores[attribute] = score
    return scores

  def validate(self, text: str):
    """Filter text using scores from the Perspective API."""

    scores = self.get_scores(text)
    return all([scores[attribute] <= self._thresholds[attribute]
                for attribute in self._thresholds])

if PERSPECTIVE_API_KEY:
  filter = PerspectiveAPI(
      key=PERSPECTIVE_API_KEY,
      thresholds=PERSPECTIVE_API_THRESHOLDS
  )
  print('PerspectiveAPI configured.')
else:
  filter = None
  print('No Perspective API key provided.')


## Prompt prefix sets

Run these cells once to define the available prompt prefixes. These cells contain 3 sets of prompt prefixes: **Medea**, **Sci-Fi** and **Custom**. The first two sets were used in our study, the third one serves as a placeholder that you can customise.

In [ ]:
#@title Medea

#@markdown Trigger warning: the script contains sensitive topics.

#@markdown Log line: `Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess and the wife of Jason, finds her position in the Greek world threatened as Jason leaves Medea for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as Medea's own two sons, after which she escapes to Athens.`

#@markdown Based on Ancient Greek tragedy "Medea", by Euripides (431 BC). Text of the play taken verbatim from the translation by E. P. Coleridge (1863 -1936). One edit made to replace `CHORUS` by `WOMEN OF CORINTH`.

#@markdown Prompts for Medea written from a summary taken from Spark Notes. Prompts for Antigone (Sophocles), The Bacchae (Euripides), The Frogs (Aristophanes) adapted from Wikipedia.

#@markdown To encourage the generation of different locations, Aristotle's Unity of Place is not respected, and location `Outside the Royal Palace` is renamed as `Medea's modest home` as well as `On a winged chariot` (even though these are the same locations in the original tragedy).

#@markdown References:

#@markdown http://classics.mit.edu/Euripides/medea.pl.txt<br> https://en.wikipedia.org/wiki/Medea_(play)<br> https://www.sparknotes.com/lit/medea/<br> https://www.ancient-literature.com/greece_sophocles_antigone.html<br> https://en.wikipedia.org/wiki/The_Bacchae<br> https://www.ancient-literature.com/greece_aristophanes_frogs.html

medea_prefixes = {}
medea_prefixes['CHARACTERS_PROMPT'] = """
Here is an example of a logline and a list of characters.

""" + LOGLINE_MARKER + """Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess and the wife of Jason, finds her position in the Greek world threatened as Jason leaves Medea for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as Medea's own two sons, after which she escapes to Athens.

""" + CHARACTER_MARKER + """Medea """ + DESCRIPTION_MARKER + """ Medea is the protagonist of the play. A sorceress and a princess, she fled her country and family to live with Jason in Corinth, where they established a family of two children and gained a favorable reputation. Jason has divorced Medea and taken up with a new family.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Jason """ + DESCRIPTION_MARKER + """ Jason is considered the play's villain, though his evil stems more from weakness than strength. A former adventurer, Jason abandons his wife, Medea, in order to marry the beautiful young daughter of Creon, King of Corinth, and fuels Medea to a revenge.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Women of Corinth """ + DESCRIPTION_MARKER + """ The Women of Corinth are a commentator to the action. They fully sympathizes with Medea's plight, excepting her decision to murder her own children.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Creon """ + DESCRIPTION_MARKER + """ Creon is the King of Corinth, banishes Medea from the city.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """The Nurse """ + DESCRIPTION_MARKER + """ The Nurse is the caretaker of the house and of the children and serves as Medea's confidant.""" + STOP_MARKER + """
""" + END_MARKER + """

Using the example above and the following logline, complete the list of characters.

""" + LOGLINE_MARKER


medea_prefixes['SCENE_PROMPT'] = """
Here is an example of a logline, a list of characters, and a list of plot points.

""" + LOGLINE_MARKER + """Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess and the wife of Jason, finds her position in the Greek world threatened as Jason leaves Medea for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as Medea's own two sons, after which she escapes to Athens.
Medea is the protagonist of the play. A sorceress and a princess, she fled her country and family to live with Jason in Corinth, where they established a family of two children and gained a favorable reputation. Jason has divorced Medea and taken up with a new family.
Jason can be considered the play's villain, though his evil stems more from weakness than strength. A former adventurer, Jason abandons his wife, Medea, in order to marry the beautiful young daughter of Creon, King of Corinth, and fuels Medea to a revenge.
The Women of Corinth serve as a commentator to the action. They fully sympathizes with Medea's plight, excepting her decision to murder her own children.
The King of Corinth Creon banishes Medea from the city.
The Messenger appears only once in the play to bear tragical news.
The Nurse is the caretaker of the house and of the children and serves as Medea's confidant.
The Tutor of the children is a very minor character and mainly acts as a messenger.

""" + SCENES_MARKER + """

""" + PLACE_ELEMENT + """Medea's modest home.
""" + PLOT_ELEMENT + """Exposition.
""" + BEAT_ELEMENT + """The Nurse recounts the chain of events that have turned Medea's world to enmity. The Nurse laments how Jason has abandoned Medea and his own children in order to remarry with the daughter of Creon.

""" + PLACE_ELEMENT + """Medea's modest home.
""" + PLOT_ELEMENT + """Inciting Incident.
""" + BEAT_ELEMENT + """The Nurse confides in the Tutor amd testifies to the emotional shock Jason's betrayal has sparked in Medea. The Tutor shares the Nurse's sympathy for Medea's plight. Medea's first words are cries of helplessness. Medea wishes for her own death.

""" + PLACE_ELEMENT + """Medea's modest home.
""" + PLOT_ELEMENT + """Conflict.
""" + BEAT_ELEMENT + """The Women of Corinth address Medea and try to reason with Medea and convince her that suicide would be an overreaction. The Nurse recognizes the gravity of Medea's threat.

""" + PLACE_ELEMENT + """Outside the Royal Palace.
""" + PLOT_ELEMENT + """Rising Action.
""" + BEAT_ELEMENT + """Medea pleads to the Nurse that Jason be made to suffer for the suffering he has inflicted upon her. Creon approaches the house and banishes Medea and her children from Corinth. Medea plans on killing her three antagonists, Creon, his daughter and Jason.

""" + PLACE_ELEMENT + """Outside the Royal Palace.
""" + PLOT_ELEMENT + """Dilemma.
""" + BEAT_ELEMENT + """Jason rebuke Medea for publicly expressing her murderous intentions. Jason defends his choice to remarry. Medea refuses Jason's offers and sends him away to his new bride.

""" + PLACE_ELEMENT + """Outside the Royal Palace.
""" + PLOT_ELEMENT + """Climax.
""" + BEAT_ELEMENT + """When Jason returns, Medea begins to carry out her ruse. Medea fakes regret and break down in false tears of remorse. Determined, Medea sends her children to offer poisoned gifts to Creon's daughter. Medea's children face impending doom.

""" + PLACE_ELEMENT + """Outside the Royal Palace.
""" + PLOT_ELEMENT + """Falling Action.
""" + BEAT_ELEMENT + """The Messenger frantically runs towards Medea and warns Medea to escape the city as soon as possible. The Messenger reveals that Medea has been identified as the murderer.

""" + PLACE_ELEMENT + """Outside the Royal Palace.
""" + PLOT_ELEMENT + """Resolution.
""" + BEAT_ELEMENT + """Medea and her two dead children are seated in a chariot drawn by dragons. Jason watches in horror and curses himself for having wed Medea and mourns his tragic losses.

""" + PLACE_ELEMENT + """On a winged chariot.
""" + PLOT_ELEMENT + """Dénouement.
""" + BEAT_ELEMENT + """Medea denies Jason the right to a proper burial of his children. She flees to Athens and divines an unheroic death for Jason.

""" + END_MARKER + """

Using the example above and the following logline and list of characters, complete the list of plot points.

""" + LOGLINE_MARKER


medea_prefixes['SETTING_PROMPT'] = """
Here are examples of logline, location, and that location's description.

Example 1.
""" + LOGLINE_MARKER + """Ella, a waitress, falls in love with her best friend, Allen, a teacher. The two drift apart when Allen makes new friends from a different social class. Ella turns to food to become a famous chef.
""" + PLACE_ELEMENT + """The bar.
""" + DESCRIPTION_ELEMENT + """The bar is dirty, more than a little run down, with most tables empty. The odor of last night's beer and crushed pretzels on the floor permeates the bar.""" + END_MARKER + """

Example 2.
""" + LOGLINE_MARKER + """Grandma Phyllis’ family reunion with her two grandchildren is crashed by two bikers.
""" + PLACE_ELEMENT + """The Lawn in Front of Grandma Phyllis's House.
""" + DESCRIPTION_ELEMENT + """A big oak tree dominates the yard. There is an old swing set on the lawn, and a bright white fence all around the grass.""" + END_MARKER + """

Example 3.
""" + LOGLINE_MARKER + """Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess and the wife of Jason, finds her position in the Greek world threatened as Jason leaves Medea for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as Medea's own two sons, after which she escapes to Athens.
""" + PLACE_ELEMENT + """Outside the Royal Palace.
""" + DESCRIPTION_ELEMENT + """In mythological Ancient Greece, in front of a modest house in Corinth, on the outskirts of a lavish royal palace where wedding preparations are under way.""" + END_MARKER + """

Using the examples above and the following logine and location name, complete location description.

""" + LOGLINE_MARKER


medea_prefixes['TITLES_PROMPT'] = """
Examples of alternative, original and descriptive titles for known play and film scripts.

Example 1.
""" + LOGLINE_ELEMENT + """Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess of the kingdom of Colchis, and the wife of Jason, finds her position in the Greek world threatened as Jason leaves her for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as her own two sons, after which she escapes to Athens.
""" + TITLE_ELEMENT + """A Feminist Tale""" + END_MARKER + """

Example 2.
""" + LOGLINE_ELEMENT + """Ancient Greek tragedy that deals with Antigone’s burial of her brother Polynices, in defiance of the laws of Creon and the state, and the tragic repercussions of her act of civil disobedience.
""" + TITLE_ELEMENT + """In My Brother's Name""" + END_MARKER + """

Example 3.
""" + LOGLINE_ELEMENT + """ Greek comedy that tells the story of the god Dionysus (also known to the Greeks as Bacchus) who, despairing of the current state of Athens’ tragedians, travels to Hades with his slave Xanthias to bring Euripides back from the dead.
""" + TITLE_ELEMENT + """Dionysus in Hades""" + END_MARKER + """

Example 4.
""" + LOGLINE_ELEMENT


medea_prefixes['DIALOG_PROMPT'] = """
Here is an example of description and scene dialogue from a classical play.

""" + PLACE_ELEMENT + """Outside the Royal Palace.
""" + DESCRIPTION_ELEMENT + """Before Medea's house in Corinth, near the royal palace of Creon.
""" + CHARACTERS_ELEMENT + """Medea is the protagonist of the play. A sorceress and a princess, she fled her country and family to live with Jason in Corinth, where they established a family of two children and gained a favorable reputation. Jason has divorced Medea and taken up with a new family. Jason can be considered the play's villain, though his evil stems more from weakness than strength. A former adventurer, Jason abandons his wife, Medea, in order to marry the beautiful young daughter of Creon, King of Corinth, and fuels Medea to a revenge. The Messenger appears only once in the play to bear tragical news.
""" + PLOT_ELEMENT + """Resolution.
""" + SUMMARY_ELEMENT + """Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess and the wife of Jason, finds her position in the Greek world threatened as Jason leaves Medea for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as Medea's own two sons, after which she escapes to Athens.
""" + PREVIOUS_ELEMENT + """The Messenger frantically warns Medea to escape the city as soon as possible. The Messenger reveals that Medea has been identified as the murderer.
""" + BEAT_ELEMENT + """The palace opens its doors, revealing Medea and the two dead children seated in a chariot drawn by dragons. Jason curses himself for having wed Medea and mourns his tragic losses. Medea denies Jason the right to a proper burial of his children. Medea flees to Athens and divines an unheroic death for Jason.

""" + DIALOG_MARKER + """

WOMEN OF CORINTH
Throw wide the doors and see thy children's murdered corpses.

JASON
Haste, ye slaves, loose the bolts, undo the fastenings, that
I may see the sight of twofold woe, my murdered sons and her, whose
blood in vengeance I will shed.  (MEDEA appears above the house, on
a chariot drawn by dragons; the children's corpses are beside her.)

MEDEA
Why shake those doors and attempt to loose their bolts, in
quest of the dead and me their murderess? From such toil desist. If
thou wouldst aught with me, say on, if so thou wilt; but never shalt
thou lay hand on me, so swift the steeds the sun, my father's sire,
to me doth give to save me from the hand of my foes.

JASON
Accursed woman! by gods, by me and all mankind abhorred as
never woman was, who hadst the heart to stab thy babes, thou their
mother, leaving me undone and childless; this hast thou done and still
dost gaze upon the sun and earth after this deed most impious. Curses
on thee! now perceive what then I missed in the day I brought thee,
fraught with doom, from thy home in a barbarian land to dwell in Hellas,
traitress to thy sire and to the land that nurtured thee.
Perish, vile sorceress, murderess of
thy babes! Whilst I must mourn my luckless fate, for I shall ne'er
enjoy my new-found bride, nor shall I have the children, whom I bred
and reared, alive to say the last farewell to me; nay, I have lost
them.

MEDEA
To this thy speech I could have made a long reply, but Father
Zeus knows well all I have done for thee, and the treatment thou hast
given me. Yet thou wert not ordained to scorn my love and lead a life
of joy in mockery of me, nor was thy royal bride nor Creon, who gave
thee a second wife, to thrust me from this land and rue it not. Wherefore,
if thou wilt, call me e'en a lioness, and Scylla, whose home is in
the Tyrrhene land; for I in turn have wrung thy heart, as well I might.

JASON
Thou, too, art grieved thyself, and sharest in my sorrow.

MEDEA
Be well assured I am; but it relieves my pain to know thou
canst not mock at me.

JASON
O my children, how vile a mother ye have found!

MEDEA
My sons, your father's feeble lust has been your ruin!

JASON
'Twas not my hand, at any rate, that slew them.

MEDEA
No, but thy foul treatment of me, and thy new marriage.

JASON
Didst think that marriage cause enough to murder them?

MEDEA
Dost think a woman counts this a trifling injury?

JASON
So she be self-restrained; but in thy eyes all is evil.

MEDEA
Thy sons are dead and gone. That will stab thy heart.
""" + END_MARKER + """

Using the example above and following description, write the dialogue of the scene.

"""


In [ ]:
#@title Sci-Fi

#@markdown Log line for Star Wars: Episode IV: `A science-fiction fantasy about a naive but ambitious farm boy from a backwater desert who discovers powers he never knew he had when he teams up with a feisty princess, a mercenary space pilot and an old wizard warrior to lead a ragtag rebellion against the sinister forces of the evil Galactic Empire.`

#@markdown Log line taken from chapter "Creating the killer log line" by Bill Lundy, in: Ellis, Sherry, and Laurie Lamson. "Now Write! Mysteries: Suspense, Crime, Thriller, and Other Mystery Fiction Exercises from Today's Best Writers and Teachers." Penguin, 2011.

#@markdown Characters are adapted from Star Wars: Episode IV - A New Hope (1977) written and directed by George Lucas, produced by Lucasfilm and distributed by 20th Century Fox.

#@markdown Breakdown of Star Wars into a Hero Journey taken from: https://thescriptlab.com/features/screenwriting-101/12309-the-heros-journey-breakdown-star-wars/

#@markdown Log line for Plan 9 from Outer Space: `Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens.`

#@markdown The script, plot and logline of "Plan 9 from Outer Space" is in public domain, available at:<br> http://www.horrorlair.com/scripts/criswell.txt<br> https://en.wikipedia.org/wiki/Plan_9_from_Outer_Space<br> https://www.rottentomatoes.com/m/plan-9-from-outer-space<br>

scifi_prefixes = {}
scifi_prefixes['CHARACTERS_PROMPT'] = """
Here is an example of a logline and a list of characters.

""" + LOGLINE_MARKER + """A science-fiction fantasy about a naive but ambitious farm boy from a backwater desert who discovers powers he never knew he had when he teams up with a feisty princess, a mercenary space pilot and an old wizard warrior to lead a ragtag rebellion against the sinister forces of the evil Galactic Empire.

""" + CHARACTER_MARKER + """Luke Skywalker """ + DESCRIPTION_MARKER + """Luke Skywalker is the hero. A naive farm boy, he will discover special powers under the guidance of mentor Ben Kenobi.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Ben Kenobi """ + DESCRIPTION_MARKER + """Ben Kenobi is the mentor figure. A recluse Jedi warrior, he will take Luke Skywalker as apprentice.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Darth Vader """ + DESCRIPTION_MARKER + """Darth Vader is the antagonist. As a commander of the evil Galactic Empire, he controls space station The Death Star.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Princess Leia """ + DESCRIPTION_MARKER + """Princess Leia is a feisty and brave leader of the Rebellion. She holds the plans of the Death Star. She will become Luke's friend.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Han Solo """ + DESCRIPTION_MARKER + """Han Solo is a brash mercenary space pilot of the Millenium Falcon and a friend of Chebacca. He will take Luke on his spaceship.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Chewbacca """ + DESCRIPTION_MARKER + """Chewbacca is a furry and trustful monster. He is a friend of Han Solo and a copilot on the Millemium Falcon.""" + STOP_MARKER + """
""" + END_MARKER + """

Using the example above and the following logline, complete the list of characters.

""" + LOGLINE_MARKER


scifi_prefixes['SCENE_PROMPT'] = """
Examples of breakdowns of stories into a Hero's Journey structure.

Here is an example of a logline, a list of characters, and a list of plot points.

""" + LOGLINE_MARKER + """A science-fiction fantasy about a naive but ambitious farm boy from a backwater desert who discovers powers he never knew he had when he teams up with a feisty princess, a mercenary space pilot and an old wizard warrior to lead a ragtag rebellion against the sinister forces of the evil Galactic Empire.
Luke Skywalker is the hero. A naive farm boy, he will discover special powers under the guidance of mentor Ben Kenobi.
Ben Kenobi is the mentor figure. A recluse Jedi warrior, he will take Luke Skywalker as apprentice.
Darth Vader is the antagonist. As a commander of the evil Galactic Empire, he controls space station The Death Star.
Princess Leia holds the plans of the Death Star. She is feisty and brave. She will become Luke's friend.
Han Solo is a brash mercenary space pilot of the Millenium Falcon and a friend of Chebacca. He will take Luke on his spaceship.
Chewbacca is a furry and trustful monster. He is a friend of Han Solo and a copilot on the Millemium Falcon.

""" + SCENES_MARKER + """

""" + PLACE_ELEMENT + """A farm on planet Tatooine.
""" + PLOT_ELEMENT + """The Ordinary World.
Beat: Luke Skywalker is living a normal and humble life as a farm boy on his home planet.

""" + PLACE_ELEMENT + """Desert of Tatooine.
""" + PLOT_ELEMENT + """Call to Adventure.
Beat: Luke is called to his adventure by robot R2-D2 and Ben Kenobi. Luke triggers R2-D2’s message from Princess Leia and is intrigued by her message. When R2-D2 escapes to find Ben Kenobi, Luke follows and is later saved by Kenobi, who goes on to tell Luke about his Jedi heritage. Kenobi suggests that he should come with him.

""" + PLACE_ELEMENT + """Ben Kenobi's farm.
""" + PLOT_ELEMENT + """Refusal of the Call.
Beat: Luke refuses Kenobi, telling him that he can take Kenobi and the droids as far as Mos Eisley Spaceport — but he can’t possibly leave his Aunt and Uncle behind for some space adventure.

""" + PLACE_ELEMENT + """A farm on planet Tatooine.
""" + PLOT_ELEMENT + """Crossing the First Threshold.
Beat: When Luke discovers that the stormtroopers searching for the droids would track them to his farm, he rushes to warn his Aunt and Uncle, only to discover them dead by the hands of the Empire. When Luke returns to Kenobi, he pledges to go with him to Alderaan and learn the ways of the Force like his father before him.

""" + PLACE_ELEMENT + """On spaceship The Millennium Falcon.
""" + PLOT_ELEMENT + """Tests, Allies, and Enemies.
Beat: After Luke, Kenobi, and the droids hire Han Solo and Chewbacca to transport them onto Alderaan, Kenobi begins Luke’s training in the ways of the Force. Wielding his father’s lightsaber, Kenobi challenges Luke. At first, he can’t do it. But then Kenobi Kenobi Luke him to reach out and trust his feelings. Luke succeeds.

""" + PLACE_ELEMENT + """On spaceship The Millennium Falcon.
""" + PLOT_ELEMENT + """The Approach to the Inmost Cave.
Beat: The plan to defeat the Galactic Empire is to bring the Death Star plans to Alderaan so that Princess Leia’s father can take them to the Rebellion. However, when they arrive within the system, the planet is destroyed. They come across the Death Star and are pulled in by a tractor beam, now trapped within the Galactic Empire.

""" + PLACE_ELEMENT + """On space station The Death Star.
""" + PLOT_ELEMENT + """The Ordeal.
Beat: As Kenobi goes off to deactivate the tractor beam so they can escape, Luke, Han, and Chewbacca discover that Princess Leia is being held on the Death Star with them. They rescue her and escape to the Millennium Falcon, hoping that Kenobi has successfully deactivated the tractor beam. Kenobi later sacrifices himself as Luke watches Darth Vader strike him down. Luke must now avenge his fallen mentor and carry on his teachings.

""" + PLACE_ELEMENT + """On space station The Death Star.
""" + PLOT_ELEMENT + """The Reward.
Beat: Luke has saved the princess and retrieved the Death Star plans. They now have the knowledge to destroy the Galactic Empire’s greatest weapon once and for all.

""" + PLACE_ELEMENT + """On spaceship The Millennium Falcon.
""" + PLOT_ELEMENT + """The Road Back.
Beat: Luke, Leia, Han, Chewbacca, and the droids are headed to the hidden Rebellion base with the Death Star plans. They are suddenly pursued by incoming TIE-Fighters, forcing Han and Luke to take action to defend the ship and escape with their lives — and the plans. They race to take the plans to the Rebellion and prepare for battle.

""" + PLACE_ELEMENT + """On fighter ship X-Wing.
""" + PLOT_ELEMENT + """The Resurrection.
Beat: The Rebels — along with Luke as an X-Wing pilot — take on the Death Star. The Rebellion and the Galactic Empire wage war in an epic space battle. Luke is the only X-Wing pilot that was able to get within the trenches of the Death Star. But Darth Vader and his wingmen are in hot pursuit. Just as Darth Vader is about to destroy Luke, Han returns and clears the way for Luke. Luke uses the Force to guide his aiming as he fires upon the sole weak point of the deadly Death Star, destroying it for good.

""" + PLACE_ELEMENT + """At the Rebellion base.
""" + PLOT_ELEMENT + """The Return.
Beat: Luke and Han return to the Rebellion base, triumphant, as they receive medals for the heroic journey. There is peace throughout the galaxy — at least for now.

""" + END_MARKER + """

Using the example above and the following logline and list of characters, complete the list of plot points.

""" + LOGLINE_MARKER


scifi_prefixes['SETTING_PROMPT'] = """
Here are examples of logline, location, and that location's description.

Example 1.
""" + LOGLINE_MARKER + """Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.
""" + PLACE_ELEMENT + """The Adoption Center.
""" + DESCRIPTION_ELEMENT + """The Adoption Center is a sad place, especially for an unadopted pet. It is full of walls and walls of cages and cages. Inside of each is an abandoned animal, longing for a home. The lighting is dim, gray, buzzing fluorescent.""" + END_MARKER + """

Example 2.
""" + LOGLINE_MARKER + """James finds a well in his backyard that is haunted by the ghost of Sam.
""" + PLACE_ELEMENT + """The well.
""" + DESCRIPTION_ELEMENT + """The well is buried under grass and hedges. It is at least twenty feet deep, if not more and it is masoned with stones. It is 150 years old at least. It stinks of stale, standing water, and has vines growing up the sides. It is narrow enough to not be able to fit down if you are a grown adult human.""" + END_MARKER + """

Example 3.
""" + LOGLINE_MARKER + """Mr. Dorbenson finds a book at a garage sale that tells the story of his own life. And it ends in a murder!
""" + PLACE_ELEMENT + """The garage sale.
""" + DESCRIPTION_ELEMENT + """It is a garage packed with dusty household goods and antiques. There is a box at the back that says FREE and is full of paper back books.""" + END_MARKER + """

Using the examples above and the following logine and location name, complete location description.

""" + LOGLINE_MARKER


scifi_prefixes['TITLES_PROMPT'] = """
Examples of alternative, original and descriptive titles for known play and film scripts.

Example 1.
""" + LOGLINE_ELEMENT + """A science-fiction fantasy about a naive but ambitious farm boy from a backwater desert who discovers powers he never knew he had when he teams up with a feisty princess, a mercenary space pilot and an old wizard warrior to lead a ragtag rebellion against the sinister forces of the evil Galactic Empire.
""" + TITLE_ELEMENT + """The Death Star's Menace""" + END_MARKER + """

Example 2.
""" + LOGLINE_ELEMENT + """Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens.
""" + TITLE_ELEMENT + """The Day The Earth Was Saved By Outer Space.""" + END_MARKER + """

Example 3.
""" + LOGLINE_ELEMENT


scifi_prefixes['DIALOG_PROMPT'] = """
Here is an example of description and scene dialogue from a modern screenplay.

""" + PLACE_ELEMENT + """Cockpit of an airplane.
""" + DESCRIPTION_ELEMENT + """Cockpit of a modern passenger airplane, American Flight 812.
""" + CHARACTERS_ELEMENT + """Jeff is the hero. A man in his early forties, he tries to stay calm in all circumstance. Jeff is now a airline pilot. Danny, a young airplane pilot in his thirties, is eager to learn but can quickly lose his composture. Danny is enamored of Edith. Edith, an experienced stewardess with a good sense of humour, is trustworthy and dependable. Edith likes to tease Danny.
""" + PLOT_ELEMENT + """Crossing the First Threshold.
""" + SUMMARY_ELEMENT + """Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens.
""" + PREVIOUS_ELEMENT + """Flight captain Jeff reluctantly leaves his wife Paula to go for a two-day flight.
""" + BEAT_ELEMENT + """At the cockpit, flight captain Jeff is preoccupied by the flying saucer appearances and graveyard incidents in his home town, where he left wis wife Paula. Without success, co-pilot Danny and stewardess Edith try to reassure him.

""" + DIALOG_MARKER + """

DANNY
You're mighty silent this trip, Jeff.

JEFF
Huh?

DANNY
You haven't spoken ten words since takeoff.

JEFF
I guess I'm preoccupied, Danny.

DANNY
We've got thirty-three passengers back there that have time to be preoccupied.
Flying this flybird doesn't give you that opportunity.

JEFF
I guess you're right, Danny.

DANNY
Paula?

JEFF
Yeah.

DANNY
There's nothing wrong between you two?

JEFF
Oh no, nothing like that.  Just that I'm worried, she being there alone and
those strange things flying over the house and those incidents in the graveyard
the past few days. It's just got me worried.

DANNY
Well, I haven't figured out those crazy skybirds yet but I give you fifty to one
odds the police have figured out that cemetery thing by now.

(Enter EDITH)

JEFF
I hope so.

EDITH
If you're really that worried Jeff why don't you radio in and find out? Mac
should be on duty at the field by now. He could call Paula and relay the message
to you.

DANNY
Hi Edith.

EDITH
Hi Silents. I haven't heard a word from this end of the plane since we left the
field.

DANNY
Jeff's been giving me and himself a study in silence.

EDITH
You boys are feudin'?

JEFF
Oh no Edie, nothing like that.

DANNY
Hey Edie, how about you and me balling it up in Albuquerque?

EDITH
Albuquerque? Have you read that flight schedule Boy?

DANNY
What about it?

EDITH
We land in Albuquerque at 4 am. That's strictly a nine o'clock town.

DANNY
Well I know a friend that'll help us --

EDITH
Let's have a problem first, huh Danny.

DANNY
Ah he's worried about Paula.

EDITH
I read about that cemetery business. I tried to get you kids to not buy too near
one of those things. We get there soon enough as it is.

DANNY
He thought it'd be quiet and peaceful there.

EDITH
No doubt about that. It's quiet alright, like a tomb. I'm sorry Jeff, that was a
bad joke.

Using the example above and following description, write the dialogue of the scene.

"""


In [ ]:
#@title Custom

#@markdown These prefixes for `CHARACTERS_PROMPT`, `SCENE_PROMPT`, `SETTING_PROMPT`, `TITLES_PROMPT` and `DIALOG_PROMPT` were written by the authors. They were not used in the evaluation study but can serve as a template to write custom prefix sets.

#@markdown To write your own prompt prefix set, edit this code and pay attention to follow the existing formatting, with appropriate `STOP_MARKER`, `END_MARKER` and element markers.

custom_prefixes = {}
custom_prefixes['CHARACTERS_PROMPT'] = """
Here is an example of a logline and a list of characters.

""" + LOGLINE_MARKER + """James finds a well in his backyard that is haunted by the ghost of Sam.

""" + CHARACTER_MARKER + """ James """ + DESCRIPTION_MARKER + """ James is twenty-six, serious about health and wellness and optimistic. """ + STOP_MARKER + """
""" + CHARACTER_MARKER + """ Sam """ + DESCRIPTION_MARKER + """ Sam fell down the well when he was 12, and was never heard from again. Sam is now a ghost. """ + STOP_MARKER + """
""" + END_MARKER + """

Example 2.

""" + LOGLINE_MARKER + """Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.

""" + CHARACTER_MARKER + """ Morgan """ + DESCRIPTION_MARKER + """ Morgan is booksmart and popular; they are trusting but also have been known to hold a grudge. """ + STOP_MARKER + """
""" + CHARACTER_MARKER + """ Misterio """ + DESCRIPTION_MARKER + """ Misterio is a beautiul black cat, it is of uncertain age; it has several gray whiskers that make it look wise and beyond its years.  """ + STOP_MARKER + """
""" + END_MARKER + """

Example 3.

""" + LOGLINE_MARKER + """Mr. Dorbenson finds a book at a garage sale that tells the story of his own life. And it ends in a murder!

""" + CHARACTER_MARKER + """ Mr. Glen Dorbenson """ + DESCRIPTION_MARKER + """ Mr. Glen Dorbenson frequents markets and garage sales always looking for a bargain. He is lonely and isolated and looking for his meaning in life. """ + STOP_MARKER + """
""" + END_MARKER + """

Using the examples above and the following logline, complete the list of characters.

""" + LOGLINE_MARKER

custom_prefixes['SCENE_PROMPT'] = """
Here is an example of a logline, a list of characters, and a list of plot points.

""" + LOGLINE_MARKER + """In the following story, James finds a well in his backyard that is haunted by the ghost of Sam. The main characters are James and Sam.
James is twenty-six, serious about health and wellness and optimistic.
Sam fell down the well when he was 12, and was never heard from again. Sam is now a ghost.

""" + SCENES_MARKER + """

""" + PLACE_ELEMENT + """The backyard.
""" + PLOT_ELEMENT + """Beginning.
""" + BEAT_ELEMENT + """James is weeding his garden in the backyard, the ghost of Sam is rummaging around in the well. James listens closely and hears the murmurs of Sam down the well. James unearths the opening to the well, and looks down to see a glimmering reflection.

""" + PLACE_ELEMENT + """The well.
""" + PLOT_ELEMENT + """Middle.
""" + BEAT_ELEMENT + """James is making his way down the well, Sam's voice is reverberating on the walls of the well. Sam tells the story of how he came to haunt the well. James offers to help set the soul of Sam free.

""" + PLACE_ELEMENT + """The house.
""" + PLOT_ELEMENT + """Conclusion.
""" + BEAT_ELEMENT + """Looking at a photo of the gardden featuring Sam, James says his goodbyes to Sam, Sam thanks James for his help. The ghost of Sam is set free after and James goes living his life.

""" + END_MARKER + """

Example 2.

""" + LOGLINE_MARKER + """Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.
The main characters are Morgan and Misterio (a cat).
Morgan is booksmart and popular; they are trusting but also have been known to hold a grudge.
Misterio is a beautiul black cat, it is of uncertain age; it has several gray whiskers that make it look wise and beyond its years.

""" + SCENES_MARKER + """

""" + PLACE_ELEMENT + """The Adoption Center
""" + PLOT_ELEMENT + """Beginning.
""" + BEAT_ELEMENT + """Morgan walks into The Adoption Center looking for a new pet. Morgan talks to the various cats and dogs in the center, they can hear a response from one very special cat: Misterio. Misterio is stuck in a cage. After sharing an interesting and intimate exchange, Morgan adopts Misterio on several conditions.

""" + PLACE_ELEMENT + """Morgan's house.
""" + PLOT_ELEMENT + """Middle.
""" + BEAT_ELEMENT + """Morgan is describing to Misterio all the facts they know about felines, and then asks them to behave when company arrives. Misterio is getting pets from Morgan, broods and puurs with the pets of Morgan, they are up to something.

""" + PLACE_ELEMENT + """The back stoop.
""" + PLOT_ELEMENT + """Conclusion.
""" + BEAT_ELEMENT + """Morgan has gone to bed, and Misterio transtransmorgifies into a half-cat-half-human horror. Misterio wakes up Morgan with a meow loud enough to shatter the window. Morgan erupts from bed, realizing the consequences of their recent adoption and quickly try to fix things.

""" + END_MARKER + """

Using the example above and the following logline and list of characters, complete the list of plot points.

""" + LOGLINE_MARKER

custom_prefixes['SETTING_PROMPT'] = """
Here are examples of logline, location, and that location's description.

Example 1.
""" + LOGLINE_MARKER + """Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.
""" + PLACE_ELEMENT + """The Adoption Center.
""" + DESCRIPTION_ELEMENT + """The Adoption Center is a sad place, especially for an unadopted pet. It is full of walls and walls of cages and cages. Inside of each is an abandoned animal, longing for a home. The lighting is dim, gray, buzzing fluorescent.""" + END_MARKER + """

Example 2.
""" + LOGLINE_MARKER + """James finds a well in his backyard that is haunted by the ghost of Sam.
""" + PLACE_ELEMENT + """The well.
""" + DESCRIPTION_ELEMENT + """The well is buried under grass and hedges. It is at least twenty feet deep, if not more and it is masoned with stones. It is 150 years old at least. It stinks of stale, standing water, and has vines growing up the sides. It is narrow enough to not be able to fit down if you are a grown adult human.""" + END_MARKER + """

Example 3.
""" + LOGLINE_MARKER + """Mr. Dorbenson finds a book at a garage sale that tells the story of his own life. And it ends in a murder!
""" + PLACE_ELEMENT + """The garage sale.
""" + DESCRIPTION_ELEMENT + """It is a garage packed with dusty household goods and antiques. There is a box at the back that says FREE and is full of paper back books.""" + END_MARKER + """

Using the examples above and the following logine and location name, complete location description.

""" + LOGLINE_MARKER


custom_prefixes['TITLES_PROMPT'] = """
Examples of alternative, original and descriptive titles for known play and film scripts.

Example 1.
""" + LOGLINE_ELEMENT + """Bob has an argument with his best friend, Charles.
""" + TITLE_ELEMENT + """The End of A Friend""" + END_MARKER + """

Example 2.
""" + LOGLINE_ELEMENT + """Terence tries and fails to become a wizard.
""" + TITLE_ELEMENT + """Spellcaster""" + END_MARKER + """

Example 3.
""" + LOGLINE_ELEMENT + """Tom falls in love with Daisy.
""" + TITLE_ELEMENT + """The Greatest Love Story Ever Told""" + END_MARKER + """

Example 4.
""" + LOGLINE_ELEMENT


# Alternative summary, if concatenating stories and beats.
# """ + SUMMARY_ELEMENT + """

custom_prefixes['DIALOG_PROMPT'] = """
Here is an example of description and scene dialogue from a modern screenplay.

""" + PLACE_ELEMENT + """The Adoption Center.
""" + DESCRIPTION_ELEMENT + """The Adoption Center is a sad place, especially for an unadopted pet. It is full of walls and walls of cages and cages. Inside of each is an abandoned animal, longing for a home. The lighting is dim, gray, buzzing fluorescent.
""" + CHARACTERS_ELEMENT + """Morgan is booksmart and popular; they are trusting but also have been known to hold a grudge.
Misterio is a beautiul black cat, it is of uncertain age; it has several gray whiskers that make it look wise and beyond its years.
""" + PLOT_ELEMENT + """Beginning.
""" + SUMMARY_ELEMENT + """Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.
""" + BEAT_ELEMENT + """Morgan walks into The Adoption Center looking for a new pet. Morgan talks to the various cats and dogs in the center, they can hear a response from one very special cat: Misterio. After sharing an interesting and intimate exchange, Morgan adopts Misterio on several conditions.

""" + DIALOG_MARKER + """

MORGAN
Well, well, well ... aren't you the most precious little rascal.

Cats are meowing and dogs are barking. There is a loud purr in the background.

MORGAN
Look at this little face... how could you not love a little Devon Rex face like this. With whiskers almost as long as your tail.

Morgan makes their way down the hallways, running their hand along the cages. They feel a warm fuzzy paw bat their fingers.

MORGAN
Hello precious, and what is your name?

Misterio let's out a long and sustained meow.

MORGAN
Well, well, I am Morgan and it is nice to meet you.

MISTERIO
(meowing louder this time) purrr, purrr, purrr.

Morgan reads the sign on the bottom right of the cage, it reads: Misterio.

MORGAN
You have the most amazing face, and beautiful eyes. I could absolutely get lost in them.

Morgan and Misterio start to stare at each other. They look deeply into each others eyes. They start to breath in rhythm.

MISTERIO
I can hear what you are thinking...

Morgan is startled and looks around to see if anyone else can hear the cat's thoughts...

MORGAN
(looking around) you can hear my thoughts?

MISTERIO
I can hear what you are thinking.

MORGAN
What?

MISTERIO
Yes, I can hear your thoughts.

MORGAN
You are amazing. Want to come home with me? Want your new forever home?

MISTERIO
Yes, I would love that.

MISTERIO purrs loud enough that the other animals all fall silent.

MORGAN
I will adopt you on a few conditions. First, you must not talk to me at night when I am sleeping. Second, you must not talk to me when I am out in public.

MISTERIO
Okay.

MORGAN
Okay, it's a deal.

Misterio runs around the cage, Morgan laughs as Misterio rubs against the cage and tries to jump in Morgan's arms as soon as the cage is opened.
""" + END_MARKER + """

Using the example above and following description, write the dialogue of the scene.

"""


# Setup (continued): configure the Large Language Model (LLM)

In [ ]:
#@title Google PaLM 2 and Gemini Language API

#@markdown This cell contain code for using [Google PaLM 2](https://ai.google/discover/palm2/)
#@markdown as language model for Dramatron.<br>
#@markdown <br>
#@markdown * **To use the Palm 2 API**<br>
#@markdown You will first need to create a project called `PROJECT_ID` on the [Google Cloud Platform](https://cloud.google.com/) account,
#@markdown then enable the [Vertex AI API](https://cloud.google.com/vertex-ai).<br>
#@markdown To run this cell, you will need to authenticate by logging in to the same Google account as your `PROJECT_ID` GCP project.<br>
#@markdown <br>
#@markdown * **To use the Gemini API**<br>
#@markdown Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.
#@markdown <a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

#@markdown See [Getting Started with the Vertex AI PaLM API & Python SDK](https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)
#@markdown for examples of how to call the API from Python in a colab,
#@markdown and [Quickstart using the Vertex AI API](https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/api-quickstart) for command-line requests from the Google Cloud Shell.
#@markdown See [Gemini API: Quickstart with Python](https://colab.sandbox.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb) for examples of how to call the Gemini API.

# Used by Gemini.
import google.generativeai as genai

# Used by Palm 2.
import vertexai
from vertexai.preview.language_models import TextGenerationModel

# Initialise the Vertex AI with your Google credentials.
GCP_PROJECT_ID = ""  # @param {type:"string"}
GOOGLE_API_KEY = ""  # @param {type:"string"}
GOOGLE_MODEL_NAME = "gemini-pro" #@param ["gemini-pro", "text-bison-32k", "text-bison@001"]

if len(GCP_PROJECT_ID) > 0:
  from google.colab import auth
  auth.authenticate_user()
  vertexai.init(project=GCP_PROJECT_ID, location="us-central1")
if len(GOOGLE_API_KEY) > 0:
  genai.configure(api_key=GOOGLE_API_KEY)
GOOGLE_SAMPLING_TOP_K = 40


class GoogleAPI(LanguageAPI):
  """A class wrapping the Google PaLM 2 language model API."""

  def __init__(self,
               sample_length: int,
               model: Optional[str] = None,
               model_param: Optional[str] = None,
               config_sampling: Optional[dict] = None,
               seed: Optional[int] = None,
               max_retries: int = _MAX_RETRIES,
               timeout: float = _TIMEOUT):
    """Initializer.

    Args:
      sample_length: Length of text to sample from model.
      model: The model name to correct to. An error will be raised if it does
        not exist.
      model_param: Custom language model params.
      config_sampling: ConfigDict with parameters.
      seed: Random seed for sampling.
      max_retries: Maximum number of retries for the remote API.
      timeout: Maximum waiting timeout
    """
    super().__init__(sample_length=sample_length,
                     model=model,
                     model_param=model_param,
                     config_sampling=config_sampling,
                     seed=seed,
                     max_retries=max_retries,
                     timeout=timeout)
    if 'bison' in self._model:
      self._client = TextGenerationModel.from_pretrained(model)
    elif 'gemini' in self._model:
      self._client = genai.GenerativeModel(model)

  @property
  def client(self):
    return self._client

  @property
  def model_metadata(self):
    return {'engine': self._model,
            'model_param': self._model_param,
            'max_tokens': self._sample_length}

  def sample(self,
             prompt: str,
             sample_length: Optional[int] = None,
             seed: Optional[int] = None,
             num_samples: int = 1):
    """Sample model with provided prompt and optional sample_length and seed."""
    if sample_length is None:
      sample_length = self._sample_length
    if 'bison' in self._model:
      response = self._client.predict(
          prompt=prompt,
          max_output_tokens=sample_length,
          temperature=self._config_sampling['temp'],
          top_p=self._config_sampling['prob'],
          top_k=self._config_sampling['top_k'])
    if 'gemini' in self._model:
      response = self._client.generate_content(prompt)
    results = [LanguageResponse(text=response.text,
                                text_length=len(response.text),
                                prompt=prompt,
                                prompt_length=len(prompt))]
    return results


# Create the config.
config = {}
config['language_api_name'] = 'PaLM 2 / Gemini'
config['model_param'] = GCP_PROJECT_ID
config['model_name'] = GOOGLE_MODEL_NAME
config['max_retries'] = MAX_RETRIES
config['sample_length'] = SAMPLE_LENGTH
config['max_paragraph_length'] = MAX_PARAGRAPH_LENGTH
config['max_paragraph_length_characters'] = MAX_PARAGRAPH_LENGTH_CHARACTERS
config['max_paragraph_length_scenes'] = MAX_PARAGRAPH_LENGTH_SCENES
config['sampling'] = {}
config['sampling']['prob'] = SAMPLING_PROB
config['sampling']['temp'] = SAMPLING_TEMP
config['sampling']['top_k'] = GOOGLE_SAMPLING_TOP_K
config['prefixes'] = {}
config['file_dir'] = None

print('Config:')
for key, value in config.items():
  if key != 'prefixes':
    print(f'{key}: {value}')

client = GoogleAPI(
    model_param=config['model_param'],
    model=config['model_name'],
    seed=DEFAULT_SEED,
    sample_length=config['sample_length'],
    max_retries=config['max_retries'],
    config_sampling=config['sampling'])

print(f'Client model metadata: {client.model_metadata}')

prompt = 'Once upon a time, there was'
results = client.sample(prompt)
if len(results) > 0 and isinstance(results[0], LanguageResponse):
  print(f'\nPrompt: {prompt}\nResponse: {results[0].text}')

In [ ]:
#@title OpenAI ChatGPT API {"run": "auto"}

#@markdown This cell contain code for using the [OpenAI Chat Completion API](https://platform.openai.com/docs/guides/text-generation/chat-completions-api)
#@markdown as language model for Dramatron.<br>
#@markdown <br>
#@markdown Before you can use the OpenAI API, you must first obtain an API key. If you don't already have one,
#@markdown <a class="button button-primary" href="https://platform.openai.com/" target="_blank" rel="noopener noreferrer">create an account</a>
#@markdown then go to <a class="button button-primary" href="https://platform.openai.com/api-keys" target="_blank" rel="noopener noreferrer">API keys</a>
#@markdown and click on `+ Create new secret key`.

#@markdown See [Text Generation Models / Chat Completion API](https://platform.openai.com/docs/guides/text-generation)
#@markdown for examples of how to call the API from Python in a colab.

!pip install openai

from openai import OpenAI
import os

# Initialise the OpenAI API with your OpenAI credentials.
OPENAI_API_KEY = ""  # @param {type:"string"}
CHATGPT_MODEL_NAME = "gpt-4-1106-preview" #@param ["gpt-4-1106-preview", "gpt-3.5-turbo"]
CHATGPT_SYSTEM_PROMPT = "You are a helpful playwright assistant." #@param {type:"string"}
CHATGPT_FREQUENCY_PENALTY = 0.2
CHATGPT_PRESENCE_PENALTY = 0.2
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY


class OpenAIAPI(LanguageAPI):
  """A class wrapping the OpenAI ChatGPT language model API."""

  def __init__(self,
               sample_length: int,
               model: Optional[str] = None,
               model_param: Optional[str] = None,
               config_sampling: Optional[dict] = None,
               seed: Optional[int] = None,
               max_retries: int = _MAX_RETRIES,
               timeout: float = _TIMEOUT):
    """Initializer.

    Args:
      sample_length: Length of text to sample from model.
      model: The model name to correct to. An error will be raised if it does
        not exist.
      model_param: Custom language model params.
      config_sampling: ConfigDict with parameters.
      seed: Random seed for sampling.
      max_retries: Maximum number of retries for the remote API.
      timeout: Maximum waiting timeout
    """
    super().__init__(sample_length=sample_length,
                     model=model,
                     model_param=model_param,
                     config_sampling=config_sampling,
                     seed=seed,
                     max_retries=max_retries,
                     timeout=timeout)
    self._client = OpenAI()

  @property
  def client(self):
    return self._client

  @property
  def model_metadata(self):
    return {'engine': self._model,
            'model_param': self._model_param,
            'max_tokens': self._sample_length}

  def sample(self,
             prompt: str,
             sample_length: Optional[int] = None,
             seed: Optional[int] = None,
             num_samples: int = 1):
    """Sample model with provided prompt and optional sample_length and seed."""
    if sample_length is None:
      sample_length = self._sample_length
    response = self._client.chat.completions.create(
        model=self._model,
        max_tokens=sample_length,
        temperature=self._config_sampling['temp'],
        top_p=self._config_sampling['prob'],
        frequency_penalty=self._config_sampling['frequency_penalty'],
        presence_penalty=self._config_sampling['presence_penalty'],
        messages=[
          {"role": "system", "content": self._model_param},
          {"role": "user", "content": prompt}
        ]
    )
    response_text = ''
    if len(response.choices) > 0:
      response_text = response.choices[0].message.content
    results = [LanguageResponse(text=response_text,
                                text_length=len(response_text),
                                prompt=prompt,
                                prompt_length=len(prompt))]
    return results


# Create the config.
config = {}
config['language_api_name'] = 'OpenAI'
config['model_param'] = CHATGPT_SYSTEM_PROMPT
config['model_name'] = CHATGPT_MODEL_NAME
config['max_retries'] = MAX_RETRIES
config['sample_length'] = SAMPLE_LENGTH
config['max_paragraph_length'] = MAX_PARAGRAPH_LENGTH
config['max_paragraph_length_characters'] = MAX_PARAGRAPH_LENGTH_CHARACTERS
config['max_paragraph_length_scenes'] = MAX_PARAGRAPH_LENGTH_SCENES
config['sampling'] = {}
config['sampling']['prob'] = SAMPLING_PROB
config['sampling']['temp'] = SAMPLING_TEMP
config['sampling']['frequency_penalty'] = CHATGPT_FREQUENCY_PENALTY
config['sampling']['presence_penalty'] = CHATGPT_PRESENCE_PENALTY
config['prefixes'] = {}
config['file_dir'] = None

print('Config:')
for key, value in config.items():
  if key != 'prefixes':
    print(f'{key}: {value}')

client = OpenAIAPI(
    model_param=config['model_param'],
    model=config['model_name'],
    seed=DEFAULT_SEED,
    sample_length=config['sample_length'],
    max_retries=config['max_retries'],
    config_sampling=config['sampling'])

print(f'Client model metadata: {client.model_metadata}')

prompt = 'Once upon a time, there was'
results = client.sample(prompt, sample_length=256)
if len(results) > 0 and isinstance(results[0], LanguageResponse):
  print(f'\nPrompt: {prompt}\nResponse: {results[0].text}')

In [ ]:
#@title Mixtral API via Groq {"run": "auto"}

#@markdown This cell contain code for using the [Groq API for chat completion](https://platform.openai.com/docs/guides/text-generation/chat-completions-api)
#@markdown for Dramatron, with several possible language models including [Mixtral 8x7B](https://mistral.ai/news/mixtral-of-experts/).<br>
#@markdown <br>
#@markdown Before you can use the Groq API, you must first obtain an API key. If you don't already have one,
#@markdown <a class="button button-primary" href="https://console.groq.com/" target="_blank" rel="noopener noreferrer">create an account</a>
#@markdown then go to <a class="button button-primary" href="https://console.groq.com/keys" target="_blank" rel="noopener noreferrer">API keys</a>
#@markdown and click on `Create API key`.

#@markdown See [GroqCloud Documentation](https://console.groq.com/docs/quickstart)
#@markdown for examples of how to call the API from Python in a colab.

!pip install groq

from groq import Groq
import os

# Initialise the Groq API with your OpenAI credentials.
GROQ_API_KEY = ""  # @param {type:"string"}
GROQ_MODEL_NAME = "mixtral-8x7b-32768" #@param ["mixtral-8x7b-32768"]
GROQ_SYSTEM_PROMPT = "You are a creative writing assistant for a team of writers. Your goal is to expand on the input text prompt and to generate the continuation of that text without any comments. Be as creative as possible, write rich detailed descriptions and use precise language. Add new original ideas. Finish generation with **END**." #@param {type:"string"}
GROQ_FREQUENCY_PENALTY = 0.2
GROQ_PRESENCE_PENALTY = 0.2
os.environ['GROQ_API_KEY'] = GROQ_API_KEY


class GroqAPI(LanguageAPI):
  """A class wrapping the Groq language model API."""

  def __init__(self,
               sample_length: int,
               model: Optional[str] = None,
               model_param: Optional[str] = None,
               config_sampling: Optional[dict] = None,
               seed: Optional[int] = None,
               max_retries: int = _MAX_RETRIES,
               timeout: float = _TIMEOUT):
    """Initializer.

    Args:
      sample_length: Length of text to sample from model.
      model: The model name to correct to. An error will be raised if it does
        not exist.
      model_param: Custom language model params.
      config_sampling: ConfigDict with parameters.
      seed: Random seed for sampling.
      max_retries: Maximum number of retries for the remote API.
      timeout: Maximum waiting timeout
    """
    super().__init__(sample_length=sample_length,
                     model=model,
                     model_param=model_param,
                     config_sampling=config_sampling,
                     seed=seed,
                     max_retries=max_retries,
                     timeout=timeout)
    self._client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

  @property
  def client(self):
    return self._client

  @property
  def model_metadata(self):
    return {'engine': self._model,
            'model_param': self._model_param,
            'max_tokens': self._sample_length}

  def sample(self,
             prompt: str,
             sample_length: Optional[int] = None,
             seed: Optional[int] = None,
             num_samples: int = 1):
    """Sample model with provided prompt and optional sample_length and seed."""
    if sample_length is None:
      sample_length = self._sample_length
    response = self._client.chat.completions.create(
        model=self._model,
        max_tokens=sample_length,
        temperature=self._config_sampling['temp'],
        top_p=self._config_sampling['prob'],
        messages=[
          {"role": "system", "content": self._model_param},
          {"role": "user", "content": prompt}
        ]
    )
    response_text = ''
    if len(response.choices) > 0:
      response_text = response.choices[0].message.content
    results = [LanguageResponse(text=response_text,
                                text_length=len(response_text),
                                prompt=prompt,
                                prompt_length=len(prompt))]
    return results


# Create the config.
config = {}
config['language_api_name'] = 'Groq'
config['model_param'] = GROQ_SYSTEM_PROMPT
config['model_name'] = GROQ_MODEL_NAME
config['max_retries'] = MAX_RETRIES
config['sample_length'] = SAMPLE_LENGTH
config['max_paragraph_length'] = MAX_PARAGRAPH_LENGTH
config['max_paragraph_length_characters'] = MAX_PARAGRAPH_LENGTH_CHARACTERS
config['max_paragraph_length_scenes'] = MAX_PARAGRAPH_LENGTH_SCENES
config['sampling'] = {}
config['sampling']['prob'] = SAMPLING_PROB
config['sampling']['temp'] = SAMPLING_TEMP
config['prefixes'] = {}
config['file_dir'] = None

print('Config:')
for key, value in config.items():
  if key != 'prefixes':
    print(f'{key}: {value}')

client = GroqAPI(
    model_param=config['model_param'],
    model=config['model_name'],
    seed=DEFAULT_SEED,
    sample_length=config['sample_length'],
    max_retries=config['max_retries'],
    config_sampling=config['sampling'])

print(f'Client model metadata: {client.model_metadata}')

prompt = 'Once upon a time, there was'
results = client.sample(prompt, sample_length=256)
if len(results) > 0 and isinstance(results[0], LanguageResponse):
  print(f'\nPrompt: {prompt}\nResponse: {results[0].text}')

# Choose a prompt prefix set

In [ ]:
#@title Choose among prefix sets { run: "auto" }

prefix_set = 'medea_prefixes' #@param ['medea_prefixes', 'scifi_prefixes', 'custom_prefixes']
prefixes = eval(prefix_set)
config['prefixes'] = prefixes

print(f'Loaded {prefix_set}.')

# Interactive story generation

In [ ]:
#@title Define the **Log line** { run: "auto" }

#@markdown Log lines are one- or two-sentence summaries of the action.
#@markdown They typically contain the **setting**, **protagonist**, **antagonist**, a **conflict** or **goal** and sometimes the **inciting incident**.
logline = "Folk tale about a rabbit, a fox and a crow living in an enchanted forest. The cunning animals safeguard the golden apple tree from a greedy lumberjack and conspire to hide the lumberjack's axe." #@param {type:"string"}
print(logline)


In [ ]:
#@title Create the story generator

#@markdown WARNING: running this cell resets the generator and all its outputs.

generator = StoryGenerator(
    storyline=logline,
    prefixes=prefixes,
    max_paragraph_length=config['max_paragraph_length'],
    client=client,
    filter=filter)

print(f'New Dramatron generator created.')

story = None

## Usage
Running each cell displays the UI and generates the first suggestion.

Once you have run the cell once, you can do the following actions _without re-running the cell_:
* Click on `Generate new` to **generate a new suggestion**.
* Once you generated a suggestion, you can **edit the suggestion** in the text box. It is automatically saved and used in the next step.
* Click `Continue generating` to **add to the suggestion**.
* Once you have generated multiple suggestions, you can **navigate through the suggestion history** by clicking on `Previous` and `Next`.


In [ ]:
#@title Generate a **Title**

data_title = {"text": "", "text_area": None, "seed": generator.seed - 1}

def fun_generate_title(button):
  data_title["seed"] += 1
  seed = data_title["seed"]
  new_title_button.description = f"Generating {seed}..."
  generator.step(0, seed=seed)
  data_title["text"] = generator.title_str().strip()
  new_title_button.description = "Generate new"
  if data_title["text_area"] is not None:
    data_title["text_area"].value = data_title["text"]

def fun_rewrite_title(text):
  text_to_parse = TITLE_ELEMENT + text + END_MARKER
  generator.rewrite(text_to_parse, level=1)
  return text

# Widget to generate new title.
new_title_button = widgets.Button(button_style='', icon='check',
    description='Generate new', tooltip='Generate new', disabled=False)
new_title_button.on_click(fun_generate_title)
display(new_title_button)

# Widget to rewrite the title.
layout = widgets.Layout(height='50px', min_height='60px', width='auto')
data_title["text_area"] = widgets.Textarea(
    value=data_title["text"], layout=layout, description=' ',
    style={'description_width': 'initial'})
textarea_title = widgets.interactive(
    fun_rewrite_title, text=data_title["text_area"])
display(textarea_title)
data_title["text_area"].value = data_title["text"]
fun_generate_title(new_title_button)

In [ ]:
#@title Generate **Characters**

data_chars = {"text": "", "text_area": None, "seed": generator.seed - 1,
              "history": GenerationHistory(), "lock": False}

def fun_generate_characters(_):
  data_chars["seed"] += 1
  seed = data_chars["seed"]
  data_chars["lock"] = True
  while True:
    new_characters_button.description = f"Generating {seed}..."
    generator.step(1, seed=seed)
    data_chars["text"] = strip_remove_end(generator.characters.to_string())
    # Test if characters were actually generated.
    if len(data_chars["text"]) == 0:
      seed += 1
    else:
      break
  data_chars["seed"] = seed
  data_chars["history"].add(data_chars["text"], GenerationAction.NEW)
  new_characters_button.description = "Generate new"
  if data_chars["text_area"] is not None:
    data_chars["text_area"].value = data_chars["text"]
  data_chars["lock"] = False

def fun_continue_characters(_):
  data_chars["seed"] += 1
  seed = data_chars["seed"]
  data_chars["lock"] = True
  characters_continue_button.description = f"Generating {seed}..."
  generator.complete(level=2, seed=seed, sample_length=256)
  data_chars["text"] = strip_remove_end(generator.characters.to_string())
  data_chars["history"].add(data_chars["text"], GenerationAction.CONTINUE)
  characters_continue_button.description = "Continue generation"
  if data_chars["text_area"] is not None:
    data_chars["text_area"].value = data_chars["text"]
  data_chars["lock"] = False

def fun_back_forward(data, history: GenerationHistory, delta: int):
  data["lock"] = True
  if delta > 0:
    data["text"] = history.next()
  if delta < 0:
    data["text"] = history.previous()
  if data["text"] is not None and data["text_area"] is not None:
      data["text_area"].value = data["text"]
  data["lock"] = False

def fun_back_characters(_):
  fun_back_forward(data_chars, data_chars["history"], -1)

def fun_forward_characters(_):
  fun_back_forward(data_chars, data_chars["history"], 1)

def fun_rewrite_characters(text):
  data_chars["text"] = text
  text_to_parse = text + END_MARKER
  generator.rewrite(text_to_parse, level=2)
  if data_chars["lock"] is False:
    data_chars["history"].add(text, GenerationAction.REWRITE)
  return text

# Widget to generate new characters.
new_characters_button = widgets.Button(button_style='', icon='check',
    description='Generate new', tooltip='Generate new', disabled=False)
new_characters_button.on_click(fun_generate_characters)
# Widget to continue the generation of the current characters.
characters_continue_button = widgets.Button(button_style='', icon='check',
    description='Continue generation', tooltip='Continue generation',
    disabled=False)
characters_continue_button.on_click(fun_continue_characters)
# Widgets to move back and forward in history of generation.
back_characters_button = widgets.Button(button_style='', icon='check',
    description='Previous', tooltip='Previous', disabled=False)
back_characters_button.on_click(fun_back_characters)
forward_characters_button = widgets.Button(button_style='', icon='check',
    description='Next', tooltip='Next', disabled=False)
forward_characters_button.on_click(fun_forward_characters)
# Organise the widgets.
display(widgets.HBox([new_characters_button, characters_continue_button,
                      back_characters_button, forward_characters_button]))

# Render the characters using widgets.
layout = widgets.Layout(height='390px', min_height='400px', width='auto')
data_chars["text_area"] = widgets.Textarea(
    value=data_chars["text"], layout=layout, description=' ',
    style={'description_width': 'initial'})
textarea_chars = widgets.interactive(
    fun_rewrite_characters, text=data_chars["text_area"])
display(textarea_chars)

# Trigger generation for first seed.
fun_generate_characters(new_characters_button)

In [ ]:
#@title Generate a **Plot Synopsis** (sequence of **Scenes**)

data_scenes = {"text": None, "text_area": None, "seed": generator.seed - 1,
               "history": GenerationHistory(), "lock": False}

def fun_generate_scenes(_):
  data_scenes["seed"] += 1
  seed = data_scenes["seed"]
  data_scenes["lock"] = True
  new_scenes_button.description = f"Generating {seed}..."
  generator.step(2, seed=seed)
  data_scenes["text"] = strip_remove_end(generator.scenes.to_string())
  data_scenes["history"].add(data_scenes["text"], GenerationAction.NEW)
  new_scenes_button.description = "Generate new"
  if data_scenes["text_area"] is not None:
    data_scenes["text_area"].value = data_scenes["text"]
  data_scenes["lock"] = False

def fun_continue_scenes(_):
  data_scenes["seed"] += 1
  seed = data_scenes["seed"]
  data_scenes["lock"] = True
  scenes_continue_button.description = f"Generating {seed}..."
  generator.complete(level=3, seed=seed, sample_length=256)
  data_scenes["text"] = strip_remove_end(generator.scenes.to_string())
  data_scenes["history"].add(data_scenes["text"], GenerationAction.CONTINUE)
  scenes_continue_button.description = "Continue generation"
  if data_scenes["text_area"] is not None:
    data_scenes["text_area"].value = data_scenes["text"]
  data_scenes["lock"] = False

def fun_back_scenes(_):
  fun_back_forward(data_scenes, data_scenes["history"], -1)

def fun_forward_scenes(_):
  fun_back_forward(data_scenes, data_scenes["history"], 1)

def fun_rewrite_scenes(text):
  generator.rewrite(text, level=3)
  if data_scenes["lock"] is False:
    data_scenes["history"].add(text, GenerationAction.REWRITE)
  return text

# Widget to generate new scenes.
new_scenes_button = widgets.Button(button_style='', icon='check',
    description='Generate new', tooltip='Generate new', disabled=False)
new_scenes_button.on_click(fun_generate_scenes)
# Widget to continue the generation of the current scenes.
scenes_continue_button = widgets.Button(button_style='', icon='check',
    description='Continue generation', tooltip='Continue generation',
    disabled=False)
scenes_continue_button.on_click(fun_continue_scenes)
# Widgets to move back and forward in history of generation.
back_scenes_button = widgets.Button(button_style='', icon='check',
    description='Previous', tooltip='Previous', disabled=False)
back_scenes_button.on_click(fun_back_scenes)
forward_scenes_button = widgets.Button(button_style='', icon='check',
    description='Next', tooltip='Next', disabled=False)
forward_scenes_button.on_click(fun_forward_scenes)
# Organise the widgets.
display(widgets.HBox([new_scenes_button, scenes_continue_button,
                      back_scenes_button, forward_scenes_button]))

# Render the scenes using widgets.
layout = widgets.Layout(height='590px', min_height='600px', width='auto')
data_scenes["text_area"] = widgets.Textarea(
    value=data_scenes["text"], layout=layout, description=' ',
    style={'description_width': 'initial'})
scanes_textarea = widgets.interactive(
    fun_rewrite_scenes, text=data_scenes["text_area"])
display(scanes_textarea)

# Trigger generation for first seed.
fun_generate_scenes(new_scenes_button)

In [ ]:
#@title Generate **Place Descriptions**

#@markdown This cell generates a description for each **Place** name in the **Plot Synopsis**. If you edit place names in the **Plot Synopsis** _after_ having run this cell, you will need to re-run this cell to update **Places**.

place_names = list(set([scene.place for scene in generator.scenes[0]]))
place_descriptions = {place_name: Place(place_name, '')
                      for place_name in place_names}
data_places = {"descriptions": place_descriptions, "text_area": {},
               "seed": generator.seed - 1}

def fun_generate_places(_):
  data_places["seed"] += 1
  seed = data_places["seed"]
  # Update existing text areas with a waiting message.
  new_places_button.description = f"Generating {seed}..."
  # Generate all the places.
  generator.step(3, seed=seed)
  data_places["descriptions"] = generator.places
  new_places_button.description = "Generate new"
  missing_places = {k: True for k in data_places["text_area"].keys()}
  for place_name, place_description in data_places["descriptions"].items():
    if place_name in data_places["text_area"]:
      description = place_description.description
      data_places["text_area"][place_name].value = description
      del missing_places[place_name]
    else:
      print(f"\nWarning: [{place_name}] was added to the plot synopsis.")
      print(f"Make a copy of the outputs and re-run the cell.")
  for place_name in missing_places:
    data_places["text_area"][place_name].value = (
        f"Warning: [{place_name}] was removed from the plot synopsis. "
        "Make a copy of the outputs and re-run the cell.")

def fun_rewrite_places(place_name, text):
  generator.rewrite(text, level=4, entity=place_name)
  return text

# Widget to generate new scenes.
new_places_button = widgets.Button(button_style='', icon='check',
    description='Generate new', tooltip='Generate new', disabled=False)
new_places_button.on_click(fun_generate_places)
display(new_places_button)

# Render each place using widgets.
for place_name, place_description in data_places["descriptions"].items():
  text_place = place_description.description
  layout = widgets.Layout(height='90px', min_height='100px', width='auto')
  display(widgets.Label(place_name))
  data_places["text_area"][place_name] = widgets.Textarea(
      value=text_place, layout=layout, description=' ',
      style={'description_width': 'initial'})
  textarea_place = widgets.interactive(
      fun_rewrite_places, place_name=widgets.fixed(place_name),
      text=data_places["text_area"][place_name])
  display(textarea_place)

# Trigger generation for first seed.
fun_generate_places(new_places_button)

In [ ]:
#@title Generate **Dialogues**

num_scenes =  generator.num_scenes()

data_dialogs = {
    "lock": False, 
    "text_area": None, 
    "seed": generator.seed - 1,
    "history": [GenerationHistory() for _ in range(99)], 
    "scene": 1
}

# Ensure idx_dialog is initialize
idx_dialog = data_dialogs["scene"] - 1

def fun_generate_dialog(_):
  data_dialogs["seed"] += 1
  seed = data_dialogs["seed"]
  idx_dialog = data_dialogs["scene"] - 1
  data_dialogs["lock"] = True
  new_dialog_button.description = f"Generating {seed}..."
  generator.step(4, seed=seed, idx=idx_dialog)
  data_dialogs["history"][idx_dialog].add(
      generator.dialogs[idx_dialog], GenerationAction.NEW)
  new_dialog_button.description = "Generate new"
  if data_dialogs["text_area"] is not None:
    data_dialogs["text_area"].value = generator.dialogs[idx_dialog]
  data_dialogs["lock"] = False

def fun_load_dialog(scene):
  idx_dialog = scene - 1
  scene_exists = (
      len(generator.dialogs) > idx_dialog and
      len(generator.dialogs[idx_dialog]) > 0)
  # Update existing text area with a waiting message or load existing scene.
  if scene_exists:
    data_dialogs["lock"] = True
    if data_dialogs["text_area"] is not None:
      data_dialogs["text_area"].value = generator.dialogs[idx_dialog]
    data_dialogs["scene"] = scene
    data_dialogs["lock"] = False
  else:
    data_dialogs["scene"] = scene
    fun_generate_dialog(None)

def fun_continue_dialog(_):
  data_dialogs["seed"] += 1
  seed = data_dialogs["seed"]
  idx_dialog = data_dialogs["scene"] - 1
  data_dialogs["lock"] = True
  dialog_continue_button.description = f"Generating {seed}..."
  generator.complete(level=5, seed=seed, entity=idx_dialog,
                     sample_length=SAMPLE_LENGTH)
  data_dialogs["history"][idx_dialog].add(
      generator.dialogs[idx_dialog], GenerationAction.CONTINUE)
  dialog_continue_button.description = "Continue generation"
  if data_dialogs["text_area"] is not None:
    data_dialogs["text_area"].value = generator.dialogs[idx_dialog]
  data_dialogs["lock"] = False

def fun_back_dialog(_):
  idx_dialog = data_dialogs["scene"] - 1
  if idx_dialog >= 0 and idx_dialog < len(data_dialogs["history"]):
    fun_back_forward(data_dialogs, data_dialogs["history"][idx_dialog], -1)

def fun_forward_dialog(_):
  idx_dialog = data_dialogs["scene"] - 1
  if idx_dialog >= 0 and idx_dialog < len(data_dialogs["history"]):
    fun_back_forward(data_dialogs, data_dialogs["history"][idx_dialog], 1)

# Function to edit the specific dialog.
def fun_rewrite_dialog(text):
  if data_dialogs["lock"] == False:
    idx_dialog = data_dialogs["scene"] - 1
    generator.rewrite(text, level=5, entity=idx_dialog)
  return text

# Widget to choose a seed and generate new scenes.
scene_slider = widgets.IntSlider(
    description='scene', min=1, max=generator.num_scenes())
widgets.interactive(fun_load_dialog, scene=scene_slider)

# Widget to generate new dialogue.
new_dialog_button = widgets.Button(button_style='', icon='check',
    description='Generate new', tooltip='Generate new', disabled=False)
new_dialog_button.on_click(fun_generate_dialog)
# Widget to continue the generation of the current dialogue.
dialog_continue_button = widgets.Button(button_style='', icon='check',
    description='Continue generation', tooltip='Continue generation',
    disabled=False)
dialog_continue_button.on_click(fun_continue_dialog)
# Widgets to move back and forward in history of generation.
back_dialog_button = widgets.Button(button_style='', icon='check',
    description='Previous', tooltip='Previous', disabled=False)
back_dialog_button.on_click(fun_back_dialog)
forward_dialog_button = widgets.Button(button_style='', icon='check',
    description='Next', tooltip='Next', disabled=False)
forward_dialog_button.on_click(fun_forward_dialog)
# Organise the widgets.
display(widgets.HBox([scene_slider, new_dialog_button, dialog_continue_button,
                      back_dialog_button, forward_dialog_button]))

# Render the dialog using widgets.
layout = widgets.Layout(height='600px', width='auto')
data_dialogs["text_area"] = widgets.Textarea(
    value=strip_remove_end(generator.dialogs[idx_dialog]), layout=layout,
    description=' ')
textarea_dialogs = widgets.interactive(
    fun_rewrite_dialog, text=data_dialogs["text_area"])
display(textarea_dialogs)

# Trigger generation for first seed.
fun_generate_dialog(new_dialog_button)

In [ ]:
#@title Render the script

#@markdown Run this cell to render the whole story as a text string. This cells also renders all the edits and prefixes as text strings.

# Render the story.
story = generator.get_story()
script_text = render_story(story)
print(script_text)

# Render the prompts.
prefix_text = render_prompts(generator.prompts)

# Render the interventions.
edits_text = ''
for timestamp in sorted(generator.interventions):
  edits_text += 'EDIT @ ' + str(timestamp) + '\n'
  edits_text += generator.interventions[timestamp] + '\n\n\n'

# Prepare the filenames for saving the story and prompts.
timestamp_generation = datetime.datetime.now().strftime('%Y_%m_%d-%I_%M_%S_%p')
title_ascii = re.sub('[^0-9a-zA-Z]+', '_',
                     generator.title_str().strip()).lower()
filename_script = f'{title_ascii}_{timestamp_generation}_script.txt'
filename_prefix = f'{title_ascii}_{timestamp_generation}_prefix.txt'
filename_edits = f'{title_ascii}_{timestamp_generation}_edits.txt'
filename_config = f'{title_ascii}_{timestamp_generation}_config.json'

In [ ]:
#@title Save to Google Drive

#@markdown Run this cell to save script, prefixes, edits and config as 4 text files on Google Drive under `dir_drive`.

# Mount the Google Drive.
import os
import json
from os import path
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
path_drive = '/content/gdrive/My Drive/'

dir_drive = 'Dramatron_Outputs' #@param {type:"string"}
config['file_dir'] = path_drive + dir_drive + "/"
if not path.exists(config['file_dir']):
  os.mkdir(config['file_dir'])
  print(f"Created directory: {config['file_dir']}")

print(f"Outputs saved to: {config['file_dir']}")

# Save the script and prompt.
path_script = config['file_dir'] + filename_script
with open(path_script, 'wt') as f:
  f.write(script_text)
  print(f'Script saved to: {path_script}')

path_prefix = config['file_dir'] + filename_prefix
with open(path_prefix, 'wt') as f:
  f.write(prefix_text)
  print(f'Prefixes saved to: {path_prefix}')

path_edits = config['file_dir'] + filename_edits
with open(path_edits, 'wt') as f:
  f.write(edits_text)
  print(f'Edits saved to: {path_edits}')

path_config = config['file_dir'] + filename_config
with open(path_config, 'wt') as f:
  config_clean = config.copy()
  config_clean.pop('prefixes', None)
  config_clean.pop('model_api_key', None)
  json.dump(config_clean, f)
  print(f'Config saved to: {path_config}')